# Import pachages

In [1]:
# Installing initial packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import awswrangler as wr
import boto3

import json
import io               
import os
from pandas import DataFrame
import datetime as dt
def install(package):
    subprocess.check_call([sys.executable, "-q", "-m", "pip", "install", package])


from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

session = boto3.Session(region_name="ap-southeast-1")
pd.options.display.max_columns=None
pd.options.display.max_rows=None

# Claims subset data - automated last two years

In [2]:
### Extracting HK, SK between a data time frame needs to be specified ### - 2 years from current

query = """
SELECT member_id,diagnosis_key from claims 
    where market in ('Singapore','Hong Kong') 
    and incurred__date__year_month_ >=(select
                                MAX(incurred__date__year_month_)-INTERVAL '2' year
                                from "hk_i_cleansed_ghb"."claims" )     
    group by member_id,diagnosis_key
"""
#setting the output of the query to a dataframe df, run this prior to running the df query
members = wr.athena.read_sql_query(query,\
#specify where the query runs                             
database="hk_i_cleansed_ghb",\
boto3_session=session,\
ctas_approach=False,\
workgroup='DataScientists')
#view
members.head()



,member_id,diagnosis_key
0,850867370:04,465.9
1,850867401:01,041.86
2,850867435:02,365.20
3,850867458:01,682.9
4,850867521:04,300.00


In [3]:
### Adjust diagnosis field ###
members['diagnosis_key_for_lookup_1'] = members['diagnosis_key'].str.replace('.', '')
members['diagnosis_key_for_lookup_2'] = members['diagnosis_key_for_lookup_1'].apply('{:0<5}'.format)

members.head()

,member_id,diagnosis_key,diagnosis_key_for_lookup_1,diagnosis_key_for_lookup_2
0,850867370:04,465.9,4659,46590
1,850867401:01,041.86,04186,04186
2,850867435:02,365.20,36520,36520
3,850867458:01,682.9,6829,68290
4,850867521:04,300.00,30000,30000


In [4]:
### Identify the MSK patients ###
members2=members[members['diagnosis_key_for_lookup_2'].isin(['71516','72210','72220','72420','72430','72840','83600','83620','95920'])].groupby(['member_id']
).agg(
members_distinct_count=('member_id', pd.Series.nunique),
).reset_index()

In [5]:
members2.head()

,member_id,members_distinct_count
0,850004357:01,1
1,850004357:02,1
2,850004965:01,1
3,850006673:01,1
4,850009036:01,1


# Two year entire Data from claims- last two years date automated

In [5]:
### Obtain claims data with entire attributes ###
#Obtain claims 01/2021-12/2022
query = """
SELECT * FROM claims 
where market in ('Singapore','Hong Kong') 
and incurred__date__year_month_ >=(select
                                  MAX(incurred__date__year_month_)-INTERVAL '2' year 
                                  from "hk_i_cleansed_ghb"."claims") 
                                  
"""

claims_df = wr.athena.read_sql_query(query,                                   
                                  database="hk_i_cleansed_ghb",
                                  boto3_session=session,
                                  ctas_approach=False,
                                  workgroup='DataScientists')

claims_df.shape
# (3308903, 95)
# (3308903, 95)


(3729557, 95)

In [7]:
claims_df.head(3)

,member_age_band,paid_to,discharge_date,member_gender,incurred__date__year_month_,diag_classification_desc,procedure_desc,claimant_age,paid_date,plan_currency,country_of_treatment,claim_original_received_date,case_type_0001,incurreddate,not_covered_amount,case_type,withhold_amount,subscriber_no,fee_amount,ccs_lv1_desc,gross_paid_amount,ailment_desc,claim_type_key,copay_incurred_currency,plan_code,scale_of_cover,allowed_amount,client_name,lf_pay_sub_flag,provider_id,incurred_yrmth,visits,group_id,environment,diagnosis_key,claims_source,paid_yrmth,form_id,received_date,member_nationality,not_covered,clinks_claim,treatment_type,copay_amount,cci_chronic,partners,chronic_flag,incurred_currency,fallout_ind,member_termination_date,claim_detail_entered_date,denied_incurred_currency,subscriber_type,product_id,group_name,network,plan_desc,billed_amount,base_currency_discount,bed_nights,doctor_name,deductable_amount,ccs_lv2_desc,subgroup_id,member_commencement_date,deductable_incurred_currency,source,person_no,member_city,allowed_incurred_currency,provider_category,diagnosis_desc,member_dob,member_location,ccs_lv3_desc,fee_amount_paid_incurred_currency,funding_type,member_id,cl_not_cov_reason,provider_name,claim_type,procedure_code,cl_not_cov_reason_desc,market,fee_incurred_currency,billed_incurred_currency,paid_date__year_month_,net_amount,non_covered_reason_key,net_incurred_currency,fee_amount_paid,incurred_date,non_covered_reason_desc,region,sys_batchdttm
0,D,Paid To Member,1900-01-01,F,2023-12-21,Factors Influencing Health Status,OUT-PATIENT DRUGS,36.0,2023-12-27,SGD,SINGAPORE,2023-12-26,NM,20231221,0.0,Medical,0.0,850867321,0.000000,Complications of pregnancy; childbirth; and th...,60.596880,Under Investigation,OP,0.0,BWG,SGP,60.596880,FACEBOOK SINGAPORE PTE LTD,<NA>,SG0001,202312,1.0,607855,DL,V22,RI,202312,61190906,2023-12-26,SRI LANKA,0.0,N,Maternity,0.0,non-chronic condition,<NA>,N,SGD,NaN,1900-01-01,2023-12-27,0.0,Subscriber,SIMT,FACEBOOK SINGAPORE PTE LTD,OON,Facebook Singapore,60.596880,0.0,0.0,<NA>,0.0,Normal pregnancy and/or delivery,001,2021-10-01,0.0,Unknown,1,SINGAPORE,80.00,SG Others,NORMAL PREGNANCY,1987-02-27,SINGAPORE,Outcome of delivery (V codes),0.0,Fully Insured,850867321:01,<NA>,SINGAPORE,OP,OPDRG,<NA>,Singapore,0.0,80.00,2023-12-27,60.596880,@,80.00,0.0,2023-12-21,nan,Asia-Pacific,13
1,E,Paid To Provider,1900-01-01,M,2023-07-20,Factors Influencing Health Status,MEDICAL TAXABLE ITEMS,40.0,2023-12-19,SGD,SINGAPORE,2023-12-07,GM,20230720,0.0,Medical,0.0,850867321,0.000000,Symptoms; signs; and ill-defined conditions an...,14.285714,Under Investigation,OP,0.0,BWG,SGP,14.285714,FACEBOOK SINGAPORE PTE LTD,N,MPSHSI,202307,1.0,607855,DL,V72.6,CX,202312,61006288,2023-12-07,UNITED KINGDOM,0.0,Y,Pure Medical,0.0,non-chronic condition,Parkway,N,SGD,NaN,1900-01-01,2023-12-19,0.0,Spouse,SIMT,FACEBOOK SINGAPORE PTE LTD,In Network,Facebook Singapore,14.285714,0.0,0.0,<NA>,0.0,Factors influencing health care,001,2021-10-01,0.0,CIGNALINKS,2,SINGAPORE,18.86,SG Others,LABORATORY EXAMINATION,1983-07-01,SINGAPORE,Medical examination/evaluation,0.0,Fully Insured,850867321:02,<NA>,PARKWAYHEALTH LABORATORY (PARKWAY LABORATORY S...,OP,TAXEM,<NA>,Singapore,0.0,18.86,2023-12-19,14.285714,@,18.86,0.0,2023-07-20,nan,Asia-Pacific,13
2,E,Paid To Provider,1900-01-01,M,2023-07-20,Factors Influencing Health Status,PATHOLOGY - OUT-PATIENT,40.0,2023-12-19,SGD,SINGAPORE,2023-12-07,GM,20230720,0.0,Medical,0.0,850867321,9.922739,Symptoms; signs; and ill-defined conditions an...,188.532043,Under Investigation,OP,0.0,BWG,SGP,178.609304,FACEBOOK SINGAPORE PTE LTD,N,MPSHSI,202307,1.0,607855,DL,V72.6,CX,202312,61006288,2023-12-07,UNITED KINGDOM,0.0,Y,Pure Medical,0.0,non-chronic condition,Parkway,N,SGD,NaN,1900-01-01,2023-12-19,0.0,Spouse,SIMT,FACEBOOK SINGAPORE PTE LTD,In Network,Facebook Singapore,198.454782,26.2,0.0,<NA>,0.0,Factors influencing health care,001,2021-10-01,0.0,CIGNALINKS,2,SINGAPORE,235.80,SG Others,LABORATORY EXAMINATION,1983-07-01,SINGAP

## Merging claims subset with claims data

In [6]:
### Merging of claims data ###
claims_data=members2[['member_id']].merge(claims_df,how ='inner',left_on=['member_id'], right_on=['member_id'])
claims_data.shape
#(765225, 95)
# (765225, 95)

(902412, 95)

In [9]:
### Check for the distinct incurred_yrmth ###
claims_data['incurred_yrmth'].value_counts().sort_index()
claims_data.head(2)

0           341
202112     4052
202201    28250
202202    25136
202203    32134
202204    32606
202205    33731
202206    34353
202207    34802
202208    37879
202209    39146
202210    39124
202211    39882
202212    38942
202301    38015
202302    40513
202303    46365
202304    42689
202305    47116
202306    42051
202307    42883
202308    45154
202309    41792
202310    43204
202311    34764
202312    17488
Name: incurred_yrmth, dtype: Int64

,member_id,member_age_band,paid_to,discharge_date,member_gender,incurred__date__year_month_,diag_classification_desc,procedure_desc,claimant_age,paid_date,plan_currency,country_of_treatment,claim_original_received_date,case_type_0001,incurreddate,not_covered_amount,case_type,withhold_amount,subscriber_no,fee_amount,ccs_lv1_desc,gross_paid_amount,ailment_desc,claim_type_key,copay_incurred_currency,plan_code,scale_of_cover,allowed_amount,client_name,lf_pay_sub_flag,provider_id,incurred_yrmth,visits,group_id,environment,diagnosis_key,claims_source,paid_yrmth,form_id,received_date,member_nationality,not_covered,clinks_claim,treatment_type,copay_amount,cci_chronic,partners,chronic_flag,incurred_currency,fallout_ind,member_termination_date,claim_detail_entered_date,denied_incurred_currency,subscriber_type,product_id,group_name,network,plan_desc,billed_amount,base_currency_discount,bed_nights,doctor_name,deductable_amount,ccs_lv2_desc,subgroup_id,member_commencement_date,deductable_incurred_currency,source,person_no,member_city,allowed_incurred_currency,provider_category,diagnosis_desc,member_dob,member_location,ccs_lv3_desc,fee_amount_paid_incurred_currency,funding_type,cl_not_cov_reason,provider_name,claim_type,procedure_code,cl_not_cov_reason_desc,market,fee_incurred_currency,billed_incurred_currency,paid_date__year_month_,net_amount,non_covered_reason_key,net_incurred_currency,fee_amount_paid,incurred_date,non_covered_reason_desc,region,sys_batchdttm
0,850004357:01,H,Paid To Member,1900-01-01,M,2023-10-10,"Symptoms, Signs, Ill-Defined Cond.",MEDICAL TAXABLE ITEMS,57.0,2023-10-12,SGD,SINGAPORE,2023-10-10,GM,20231010,0.0,Medical,0.0,850004357,0.0,Diseases of the circulatory system,2.277706,Neoplasms,OP,0.0,CQW,SGP,2.277706,LGT BANK (SINGAPORE) LTD,<NA>,U98731,202310,1.0,610252,DL,785.1,RI,202310,60376750,2023-10-10,SWITZERLAND,0.0,N,Pure Medical,0.0,non-chronic condition,<NA>,N,SGD,NaN,1900-01-01,2023-10-12,0.0,Subscriber,SIMT,LGT BANK (SINGAPORE) LTD,OON,LGT Bank Directors,2.277706,0.0,0.0,<NA>,0.0,Diseases of the heart,001,2002-07-01,0.0,Unknown,1,SINGAPORE,3.12,SG Others,PALPITATIONS,1966-07-22,SINGAPORE,Cardiac dysrhythmias,0.0,Fully Insured,<NA>,THE MING CLINIC,OP,TAXEM,<NA>,Singapore,0.0,3.12,2023-10-12,2.277706,@,3.12,0.0,2023-10-10,nan,Asia-Pacific,30
1,850004357:01,H,Paid To Member,1900-01-01,M,2023-10-10,"Symptoms, Signs, Ill-Defined Cond.",OUT-PATIENT DRUGS,57.0,2023-10-12,SGD,SINGAPORE,2023-10-10,GM,20231010,0.0,Medical,0.0,850004357,0.0,Diseases of the circulatory system,28.471326,Neoplasms,OP,0.0,CQW,SGP,28.471326,LGT BANK (SINGAPORE) LTD,<NA>,U98731,202310,1.0,610252,DL,785.1,RI,202310,60376750,2023-10-10,SWITZERLAND,0.0,N,Pure Medical,0.0,non-chronic condition,<NA>,N,SGD,NaN,1900-01-01,2023-10-12,0.0,Subscriber,SIMT,LGT BANK (SINGAPORE) LTD,OON,LGT Bank Directors,28.471326,0.0,0.0,<NA>,0.0,Diseases of the heart,001,2002-07-01,0.0,Unknown,1,SINGAPORE,39.00,SG Others,PALPITATIONS,1966-07-22,SINGAPORE,Cardiac dysrhythmias,0.0,Fully Insured,<NA>,THE MING CLINIC,OP,OPDRG,<NA>,Singapore,0.0,39.00,2023-10-12,28.471326,@,39.00,0.0,2023-10-10,nan,Asia-Pacific,30


# Eligibility last two years - date automated

In [7]:
### Extracting of eligibilty data to get wide range of demographical attributes ### : (dates needs to mentioned)

query = """
SELECT 
group_industry_sector_desc, plan_code, member_family_type, rider_5, type, member_gender, eap, group_name, subscriber_no, funding_type, group_id, member_nationality, vision, wellness, client_location, eligible_mth, dental, eligible_yrmth, person_no, member_location, member_city, aoc, eligible_yr, subscriber_type,
member_age, policy_commencement, client_name, product_id, product_description,
member_dob, member_id, subgroup_code
from eligibility 
where eligible_date >=(SELECT 
                    (MAX(eligible_date)- INTERVAL '2' year) as date_new
                     from "hk_i_cleansed_ghb"."eligibility")     
group by
group_industry_sector_desc, plan_code, member_family_type, rider_5, type, member_gender, eap, group_name, subscriber_no, funding_type, group_id, member_nationality, vision, wellness, client_location, eligible_mth, dental, eligible_yrmth, person_no, member_location, member_city, aoc, eligible_yr, subscriber_type,
member_age, policy_commencement, client_name, product_id, product_description,
member_dob, member_id, subgroup_code
"""
#setting the output of the query to a dataframe df, run this prior to running the df query
elig = wr.athena.read_sql_query(query,\
#specify where the query runs                             
database="hk_i_cleansed_ghb",\
boto3_session=session,\
ctas_approach=False,\
workgroup='DataScientists')
#view
elig.head()

### Merge eligibility data for MSK patients ###
elig.shape
# (1619657, 32)
elig_data=members2[['member_id']].merge(elig,how ='inner',left_on=['member_id'], right_on=['member_id'])
elig_data.shape

# (182628, 32)

,group_industry_sector_desc,plan_code,member_family_type,rider_5,type,member_gender,eap,group_name,subscriber_no,funding_type,group_id,member_nationality,vision,wellness,client_location,eligible_mth,dental,eligible_yrmth,person_no,member_location,member_city,aoc,eligible_yr,subscriber_type,member_age,policy_commencement,client_name,product_id,product_description,member_dob,member_id,subgroup_code
0,"Banking,Investment & Insurance (inc.Brokers,Ag...",DTA,F,nan,US EXPAT/Inpat/National,M,Yes,GS PREMIER HEALTHCARE PLAN,850891272,Fully Insured,609294,UNITED STATES,Yes,No,Hong Kong,12,Yes,202312,1,SINGAPORE,SINGAPORE,W/W,2023,Employee,46,2022-09-01,GOLDMAN SACHS (ASIA) L.L.C.,HKMT,HK MED MID TIER,1977-03-04,850891272:01,010
1,Miscellaneous Services,DBY,F,nan,TCN,M,Yes,DISCOVERMARKET ASIA PTE LTD,850891292,Fully Insured,612729,INDIA,Yes,No,Singapore,12,Yes,202312,4,SINGAPORE,SINGAPORE,SGP,2023,Dependant,4,2023-10-01,DISCOVERMARKET ASIA PTE LTD,SIMT,SGP REG MID TIER,2019-05-08,850891292:04,001
2,Communications & Computing,BDX,C,nan,TCN,F,Yes,SPLUNK SERVICES SINGAPORE PTE. LTD.,850891372,Fully Insured,606740,JAPAN,No,No,Singapore,12,Yes,202312,2,SINGAPORE,SINGAPORE,SGP,2023,Spouse,47,2020-11-08,SPLUNK SERVICES SINGAPORE PTE. LTD.,SIMT,SGP REG MID TIER,1976-10-30,850891372:02,001
3,Communications & Computing,BWG,C,nan,US EXPAT/Inpat/National,F,Yes,FACEBOOK SINGAPORE PTE LTD,850892012,Fully Insured,607855,UNITED STATES,No,No,Singapore,12,Yes,202312,2,SINGAPORE,SINGAPORE,SGP,2023,Spouse,36,2021-10-01,FACEBOOK SINGAPORE PTE LTD,SIMT,SGP REG MID TIER,1987-03-30,850892012:02,001
4,Miscellaneous Services,COW,C,nan,KLN,M,Yes,INFORMATICA S.E.A. PTE LTD,850892125,Fully Insured,606029,SINGAPORE,Yes,No,Singapore,12,Yes,202312,1,SINGAPORE,CENTRAL SINGAPORE,SGP,2023,Employee,46,2020-01-01,INFORMATICA S.E.A. PTE LTD,SIMT,SGP REG MID TIER,1977-01-21,850892125:01,001


(1834592, 32)

(220329, 32)

In [11]:
### Checking for thr distribution of eligible year month ###
elig_data['eligible_yrmth'].value_counts().sort_index()

202112    7791
202201    8073
202202    8171
202203    8297
202204    8420
202205    8573
202206    8698
202207    8851
202208    8997
202209    9075
202210    9099
202211    9147
202212    9217
202301    9408
202302    9378
202303    9302
202304    9249
202305    9125
202306    9013
202307    8974
202308    8886
202309    8811
202310    8672
202311    8548
202312    8554
Name: eligible_yrmth, dtype: Int64

## Merge eligibitily data with claims 

In [8]:
### Link eligibility data to claims data ###
elig_claim=elig_data.merge(claims_data,how ='left',left_on=['member_id', 'eligible_yrmth','group_id'], right_on=['member_id','incurred_yrmth','group_id'])
elig_claim.shape

# (830003, 125)
elig_claim.head()
### Drop redundent fields ###
elig_claim.drop(['member_city_y','group_name_y','plan_code_y','subscriber_no_y','client_name_y','funding_type_y','subscriber_type_y','member_location_y','person_no_y','product_id_y','member_gender_y','member_nationality_y','member_dob_y'], axis=1, inplace=True)
elig_claim = elig_claim.rename(columns={'plan_code_x':'plan_code','member_gender_x':'member_gender','group_name_x':'group_name','subscriber_no_x':'subscriber_no','funding_type_x':'funding_type','member_nationality_x':'member_nationality','person_no_x':'person_no','member_location_x':'member_location','member_city_x':'member_city','subscriber_type_x':'subscriber_type','client_name_x':'client_name','product_id_x':'product_id','member_dob_x':'member_dob'})
elig_claim.head()

#### Check the data ####
elig_claim.shape

# (830003, 112)

### Unwanted columns removed and drop duplicates ###
elig_claim_v2 = elig_claim.drop_duplicates()
print(" After removing duplicates:",elig_claim_v2.shape)

#  After removing duplicates: (829999, 112)

(983138, 125)

,member_id,group_industry_sector_desc,plan_code_x,member_family_type,rider_5,type,member_gender_x,eap,group_name_x,subscriber_no_x,funding_type_x,group_id,member_nationality_x,vision,wellness,client_location,eligible_mth,dental,eligible_yrmth,person_no_x,member_location_x,member_city_x,aoc,eligible_yr,subscriber_type_x,member_age,policy_commencement,client_name_x,product_id_x,product_description,member_dob_x,subgroup_code,member_age_band,paid_to,discharge_date,member_gender_y,incurred__date__year_month_,diag_classification_desc,procedure_desc,claimant_age,paid_date,plan_currency,country_of_treatment,claim_original_received_date,case_type_0001,incurreddate,not_covered_amount,case_type,withhold_amount,subscriber_no_y,fee_amount,ccs_lv1_desc,gross_paid_amount,ailment_desc,claim_type_key,copay_incurred_currency,plan_code_y,scale_of_cover,allowed_amount,client_name_y,lf_pay_sub_flag,provider_id,incurred_yrmth,visits,environment,diagnosis_key,claims_source,paid_yrmth,form_id,received_date,member_nationality_y,not_covered,clinks_claim,treatment_type,copay_amount,cci_chronic,partners,chronic_flag,incurred_currency,fallout_ind,member_termination_date,claim_detail_entered_date,denied_incurred_currency,subscriber_type_y,product_id_y,group_name_y,network,plan_desc,billed_amount,base_currency_discount,bed_nights,doctor_name,deductable_amount,ccs_lv2_desc,subgroup_id,member_commencement_date,deductable_incurred_currency,source,person_no_y,member_city_y,allowed_incurred_currency,provider_category,diagnosis_desc,member_dob_y,member_location_y,ccs_lv3_desc,fee_amount_paid_incurred_currency,funding_type_y,cl_not_cov_reason,provider_name,claim_type,procedure_code,cl_not_cov_reason_desc,market,fee_incurred_currency,billed_incurred_currency,paid_date__year_month_,net_amount,non_covered_reason_key,net_incurred_currency,fee_amount_paid,incurred_date,non_covered_reason_desc,region,sys_batchdttm
0,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",ATV,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,605896,SWITZERLAND,Yes,No,Singapore,6,Yes,202206,1,SINGAPORE,SINGAPORE,SGP,2022,Employee,57,2020-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,<NA>,<NA>,NaT,<NA>,NaT,<NA>,<NA>,NaN,NaT,<NA>,<NA>,NaT,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaT,NaT,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaT,NaN,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,NaT,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaT,NaN,<NA>,NaN,NaN,NaT,<NA>,<NA>,<NA>
1,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",CQW,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,610252,SWITZERLAND,Yes,No,Singapore,12,Yes,202312,1,SINGAPORE,SINGAPORE,SGP,2023,Employee,57,2023-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,H,Paid To Member,1900-01-01,M,2023-12-04,Factors Influencing Health Status,VISION - LENSES,57.0,2023-12-11,SGD,SINGAPORE,2023-12-04,VI,20231204,0.0,Vision,0.0,850004357,0.0,Diseases of the nervous system and sense organs,1439.175900,Under Investigation,OP,0.0,CQW,SGP,1439.175900,LGT BANK (SINGAPORE) LTD,<NA>,Y02670,202312,1.0,DL,V72.0,RI,202312,60934006,2023-12-04,SWITZERLAND,0.0,N,Vision,0.0,non-chronic condition,<NA>,N,SGD,NaN,1900-01-01,2023-12-05,0.0,Subscriber,SIMT,LGT BANK (SINGAPORE) LTD,OON,LGT Bank Directors,1439.175900,0.0,0.0,<NA>,0.0,Eye disorders,001,2002-07-01,0.0,Unknown,1,SINGAPORE,1900.0,SG Others,EYE,1966-07-22,SINGAPORE,Other eye disorders,0.0,Fully Insured,<NA>,EYE WERKS PTE LTD,OP,LENSE,<NA>,Singapore,0.0,1900.0,2023-12-11,1439.175900,@,1900.0,0.0,2023-12-04,nan,Asia-Pacific,4
2,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",CQW,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,610252,SWITZERLAND,Yes,No,Singapore,12,Yes,202312,1,SINGAPORE,SINGAPORE,SGP,2023,Employee,57,2023-01-01,LGT BANK (SING

,member_id,group_industry_sector_desc,plan_code,member_family_type,rider_5,type,member_gender,eap,group_name,subscriber_no,funding_type,group_id,member_nationality,vision,wellness,client_location,eligible_mth,dental,eligible_yrmth,person_no,member_location,member_city,aoc,eligible_yr,subscriber_type,member_age,policy_commencement,client_name,product_id,product_description,member_dob,subgroup_code,member_age_band,paid_to,discharge_date,incurred__date__year_month_,diag_classification_desc,procedure_desc,claimant_age,paid_date,plan_currency,country_of_treatment,claim_original_received_date,case_type_0001,incurreddate,not_covered_amount,case_type,withhold_amount,fee_amount,ccs_lv1_desc,gross_paid_amount,ailment_desc,claim_type_key,copay_incurred_currency,scale_of_cover,allowed_amount,lf_pay_sub_flag,provider_id,incurred_yrmth,visits,environment,diagnosis_key,claims_source,paid_yrmth,form_id,received_date,not_covered,clinks_claim,treatment_type,copay_amount,cci_chronic,partners,chronic_flag,incurred_currency,fallout_ind,member_termination_date,claim_detail_entered_date,denied_incurred_currency,network,plan_desc,billed_amount,base_currency_discount,bed_nights,doctor_name,deductable_amount,ccs_lv2_desc,subgroup_id,member_commencement_date,deductable_incurred_currency,source,allowed_incurred_currency,provider_category,diagnosis_desc,ccs_lv3_desc,fee_amount_paid_incurred_currency,cl_not_cov_reason,provider_name,claim_type,procedure_code,cl_not_cov_reason_desc,market,fee_incurred_currency,billed_incurred_currency,paid_date__year_month_,net_amount,non_covered_reason_key,net_incurred_currency,fee_amount_paid,incurred_date,non_covered_reason_desc,region,sys_batchdttm
0,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",ATV,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,605896,SWITZERLAND,Yes,No,Singapore,6,Yes,202206,1,SINGAPORE,SINGAPORE,SGP,2022,Employee,57,2020-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,<NA>,<NA>,NaT,NaT,<NA>,<NA>,NaN,NaT,<NA>,<NA>,NaT,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaT,NaT,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaT,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaT,NaN,<NA>,NaN,NaN,NaT,<NA>,<NA>,<NA>
1,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",CQW,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,610252,SWITZERLAND,Yes,No,Singapore,12,Yes,202312,1,SINGAPORE,SINGAPORE,SGP,2023,Employee,57,2023-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,H,Paid To Member,1900-01-01,2023-12-04,Factors Influencing Health Status,VISION - LENSES,57.0,2023-12-11,SGD,SINGAPORE,2023-12-04,VI,20231204,0.0,Vision,0.0,0.0,Diseases of the nervous system and sense organs,1439.175900,Under Investigation,OP,0.0,SGP,1439.175900,<NA>,Y02670,202312,1.0,DL,V72.0,RI,202312,60934006,2023-12-04,0.0,N,Vision,0.0,non-chronic condition,<NA>,N,SGD,NaN,1900-01-01,2023-12-05,0.0,OON,LGT Bank Directors,1439.175900,0.0,0.0,<NA>,0.0,Eye disorders,001,2002-07-01,0.0,Unknown,1900.0,SG Others,EYE,Other eye disorders,0.0,<NA>,EYE WERKS PTE LTD,OP,LENSE,<NA>,Singapore,0.0,1900.0,2023-12-11,1439.175900,@,1900.0,0.0,2023-12-04,nan,Asia-Pacific,4
2,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",CQW,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,610252,SWITZERLAND,Yes,No,Singapore,12,Yes,202312,1,SINGAPORE,SINGAPORE,SGP,2023,Employee,57,2023-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,H,Paid To Member,1900-01-01,2023-12-08,Infectious & Parasitic Diseases,MEDICAL TAXABLE ITEMS,57.0,2023-12-12,SGD,SINGAPORE,2023-12-08,GM,20231208,0.0,Medical,0.0,0.0,Infectious and parasitic diseases,3.635813,Infectious & Parasitic Diseases,OP,0.0,SGP,3.635813,<NA>,U98731,202312,1.0,DL,079.82,RI,202312,61018692,2023-12-08,0.0,N,Pure Medical,0.0,non-chronic condition,<NA>,N,SG

(983138, 112)

 After removing duplicates: (983134, 112)


In [9]:
### Minor Exlusion from the analysis ###
msk_elig_claim = elig_claim_v2.loc[(elig_claim_v2['member_age'] >= 18)]
print(" msk_elig_claim table shape:",msk_elig_claim.shape)

# msk_elig_claim table shape: (809258, 112)

#Distinct member count from the overall table:
msk_elig_claim['member_id'].nunique()
# 9554

## Deriving a column visits_derived from incurred_date and member_id ###
msk_elig_claim['incurred_date'] = pd.to_datetime(msk_elig_claim['incurred_date'])
msk_elig_claim['visits_derived'] = msk_elig_claim['member_id'].astype(str)+'_'+ msk_elig_claim['incurred_date'].dt.date.astype(str)
msk_elig_claim.head(3)

 msk_elig_claim table shape: (958723, 112)


10339

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,member_id,group_industry_sector_desc,plan_code,member_family_type,rider_5,type,member_gender,eap,group_name,subscriber_no,funding_type,group_id,member_nationality,vision,wellness,client_location,eligible_mth,dental,eligible_yrmth,person_no,member_location,member_city,aoc,eligible_yr,subscriber_type,member_age,policy_commencement,client_name,product_id,product_description,member_dob,subgroup_code,member_age_band,paid_to,discharge_date,incurred__date__year_month_,diag_classification_desc,procedure_desc,claimant_age,paid_date,plan_currency,country_of_treatment,claim_original_received_date,case_type_0001,incurreddate,not_covered_amount,case_type,withhold_amount,fee_amount,ccs_lv1_desc,gross_paid_amount,ailment_desc,claim_type_key,copay_incurred_currency,scale_of_cover,allowed_amount,lf_pay_sub_flag,provider_id,incurred_yrmth,visits,environment,diagnosis_key,claims_source,paid_yrmth,form_id,received_date,not_covered,clinks_claim,treatment_type,copay_amount,cci_chronic,partners,chronic_flag,incurred_currency,fallout_ind,member_termination_date,claim_detail_entered_date,denied_incurred_currency,network,plan_desc,billed_amount,base_currency_discount,bed_nights,doctor_name,deductable_amount,ccs_lv2_desc,subgroup_id,member_commencement_date,deductable_incurred_currency,source,allowed_incurred_currency,provider_category,diagnosis_desc,ccs_lv3_desc,fee_amount_paid_incurred_currency,cl_not_cov_reason,provider_name,claim_type,procedure_code,cl_not_cov_reason_desc,market,fee_incurred_currency,billed_incurred_currency,paid_date__year_month_,net_amount,non_covered_reason_key,net_incurred_currency,fee_amount_paid,incurred_date,non_covered_reason_desc,region,sys_batchdttm,visits_derived
0,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",ATV,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,605896,SWITZERLAND,Yes,No,Singapore,6,Yes,202206,1,SINGAPORE,SINGAPORE,SGP,2022,Employee,57,2020-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,<NA>,<NA>,NaT,NaT,<NA>,<NA>,NaN,NaT,<NA>,<NA>,NaT,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaT,NaT,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaT,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaT,NaN,<NA>,NaN,NaN,NaT,<NA>,<NA>,<NA>,850004357:01_NaT
1,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",CQW,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,610252,SWITZERLAND,Yes,No,Singapore,12,Yes,202312,1,SINGAPORE,SINGAPORE,SGP,2023,Employee,57,2023-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,H,Paid To Member,1900-01-01,2023-12-04,Factors Influencing Health Status,VISION - LENSES,57.0,2023-12-11,SGD,SINGAPORE,2023-12-04,VI,20231204,0.0,Vision,0.0,0.0,Diseases of the nervous system and sense organs,1439.175900,Under Investigation,OP,0.0,SGP,1439.175900,<NA>,Y02670,202312,1.0,DL,V72.0,RI,202312,60934006,2023-12-04,0.0,N,Vision,0.0,non-chronic condition,<NA>,N,SGD,NaN,1900-01-01,2023-12-05,0.0,OON,LGT Bank Directors,1439.175900,0.0,0.0,<NA>,0.0,Eye disorders,001,2002-07-01,0.0,Unknown,1900.0,SG Others,EYE,Other eye disorders,0.0,<NA>,EYE WERKS PTE LTD,OP,LENSE,<NA>,Singapore,0.0,1900.0,2023-12-11,1439.175900,@,1900.0,0.0,2023-12-04,nan,Asia-Pacific,4,850004357:01_2023-12-04
2,850004357:01,"Banking,Investment & Insurance (inc.Brokers,Ag...",CQW,F,nan,TCN,M,Yes,LGT BANK (SINGAPORE) LTD,850004357,Fully Insured,610252,SWITZERLAND,Yes,No,Singapore,12,Yes,202312,1,SINGAPORE,SINGAPORE,SGP,2023,Employee,57,2023-01-01,LGT BANK (SINGAPORE) LTD,SIMT,SGP REG MID TIER,1966-07-22,001,H,Paid To Member,1900-01-01,2023-12-08,Infectious & Parasitic Diseases,MEDICAL TAXABLE ITEMS,57.0,2023-12-12,SGD,SINGAPORE,2023-12-08,GM,20231208,0.0,Medical,0.0,0.0,Infectious and parasitic diseases,3.635813,Infectious & Parasitic Diseases,OP,0.0,SGP,3.635813,<NA>,U98731,202312,1.0,DL,079.82,RI,202312,61018692,2023-12-0

In [14]:
msk_elig_claim.dtypes

member_id                                    object
group_industry_sector_desc                   string
plan_code                                    string
member_family_type                           string
rider_5                                      string
type                                         string
member_gender                                string
eap                                          string
group_name                                   string
subscriber_no                                string
funding_type                                 string
group_id                                      Int32
member_nationality                           string
vision                                       string
wellness                                     string
client_location                              string
eligible_mth                                  Int32
dental                                       string
eligible_yrmth                                Int32
person_no   

# Observation period 1 year - date automated

In [10]:
msk_elig_claim_obs=msk_elig_claim.loc[(msk_elig_claim['incurred__date__year_month_']>=(msk_elig_claim['incurred__date__year_month_'].max()-pd.DateOffset(months=15)))
                   & (msk_elig_claim['incurred__date__year_month_']<=(msk_elig_claim['incurred__date__year_month_'].max()-
                                                                     pd.DateOffset(months=3))) ]


In [11]:
obs_start_date=msk_elig_claim['incurred__date__year_month_'].max()-pd.DateOffset(months=15) 
obs_end_date=msk_elig_claim['incurred__date__year_month_'].max()-pd.DateOffset(months=3) 
print(obs_start_date)
print(obs_end_date)


2022-09-28 00:00:00
2023-09-28 00:00:00


In [12]:
# Automate to generate the latest csv for modeling
wr.s3.to_csv(
    df=msk_elig_claim_obs,
    path='s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/OBSERVATION_Period.csv') 

{'paths': ['s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/OBSERVATION_Period.csv'],
 'partitions_values': {}}

In [13]:
# ### Extract the observation period ###
# msk_elig_claim_obs = msk_elig_claim.loc[(msk_elig_claim['incurred_yrmth'] >= 202201) & (msk_elig_claim['incurred_yrmth'] <= 202212)]
 
print("Distinct Member claims in 12 month period:",msk_elig_claim_obs['member_id'].nunique())
# Distinct Member claims in 12 month period: 8745

# Adding manipulated diagnosis_key column #
msk_elig_claim_obs['diagnosis_key_for_lookup_1'] = msk_elig_claim_obs['diagnosis_key'].str.replace('.', '')
msk_elig_claim_obs['diagnosis_key_for_lookup_2'] = msk_elig_claim_obs['diagnosis_key_for_lookup_1'].apply('{:0<5}'.format)

### Getting eligible month member count ###
elig_member_count_df = msk_elig_claim_obs.groupby(['member_id']).agg(
    eligible_yrmth_count=('eligible_yrmth', 'nunique')
)
elig_member_count_df = elig_member_count_df.reset_index()
elig_member_count_df.head()
elig_member_count_df.shape

# (8745, 2)


### Processing member level data with total paid amount and visit_count summed ### ! Make sure to name visits_counts as total_visits_count
grouped_msk = msk_elig_claim_obs.groupby(['member_id', 'market','member_location','member_age','member_gender','claim_type',]).agg(
    total_paid_amount=('gross_paid_amount', 'sum'),
    visits_count=('visits_derived', 'nunique')
)
grouped_msk = grouped_msk.reset_index()
grouped_msk.shape
# (10281, 8)

### Merge the eligible month count dataframe and grouped dataframe ###
elig_grouped_msk=pd.merge(grouped_msk, elig_member_count_df, left_on='member_id', right_on='member_id', how='inner')
elig_grouped_msk.shape

# 10281, 9)
elig_grouped_msk.head()

Distinct Member claims in 12 month period: 9651


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,member_id,eligible_yrmth_count
0,850004357:01,8
1,850004357:02,12
2,850004965:01,5
3,850012059:02,5
4,850143894:01,10


(9651, 2)

(11464, 8)

(11464, 9)

,member_id,market,member_location,member_age,member_gender,claim_type,total_paid_amount,visits_count,eligible_yrmth_count
0,850004357:01,Singapore,SINGAPORE,57,M,OP,4331.169672,14,8
1,850004357:02,Singapore,SINGAPORE,56,F,OP,14455.761894,57,12
2,850004965:01,Hong Kong,MALAYSIA,58,M,OP,1926.120000,6,5
3,850012059:02,Singapore,SINGAPORE,47,F,OP,1332.140000,8,5
4,850143894:01,Singapore,SINGAPORE,38,F,OP,4598.690000,24,10


In [14]:
# Check the number of member_ids who had duplicate rows
# Count the number of duplicate distinct member_ids
duplicate_count = elig_grouped_msk.duplicated('member_id').sum()
print("Number of duplicate distinct members:", duplicate_count)

# Number of duplicate distinct members: 1536

Number of duplicate distinct members: 1813


In [15]:
#### Implementing logic to market with highest health care visits affliated to a member ###
# Calculate the total_visits counts for each member and market
total_visits = elig_grouped_msk.groupby(['member_id','market','eligible_yrmth_count'])['visits_count'].sum().reset_index()

# Find the market with highest visit care counts for each member
highest_visit_market = total_visits.groupby('member_id')['visits_count'].idxmax()
highest_visit_market.head()

#Create a new dataframe from selected market values and total_visits
msk_claim_visits_df = total_visits.loc[highest_visit_market,['member_id','market','visits_count','eligible_yrmth_count']]
msk_claim_visits_df.reset_index(drop= True, inplace=True)

### Distinguish OP and IP claim total paid amount ###
msk_claims_amt_df = elig_grouped_msk.pivot_table('total_paid_amount',['member_id','member_location','member_age','member_gender'],'claim_type')

# Removing the indexing and renaming the axis
msk_claims_amt_df.reset_index(inplace=True)
msk_claims_amt_df = msk_claims_amt_df.rename_axis(None,axis=1)
column_mapping = {'IP':'IP_total_paid_amount','OP':'OP_total_paid_amount'}
msk_claims_amt_df.rename(columns=column_mapping,inplace=True)

# Fill NaN values with 0 
msk_claims_amt_df['IP_total_paid_amount'].fillna(0,inplace=True)
msk_claims_amt_df['OP_total_paid_amount'].fillna(0,inplace=True)
msk_claims_amt_df.head()
msk_claims_amt_df.shape

#(8745, 6)
# No dup found!

### Distincguish OP and IP visits ###
msk_claims_visits_df = elig_grouped_msk.pivot_table('visits_count',['member_id','member_location','member_age','member_gender'],'claim_type')

# Removing the indexing and renaming the axis
msk_claims_visits_df.reset_index(inplace=True)
msk_claims_visits_df = msk_claims_visits_df.rename_axis(None,axis=1)
column_mapping = {'IP':'IP_visits_count','OP':'OP_visits_count'}
msk_claims_visits_df.rename(columns=column_mapping,inplace=True)

# Fill NaN values with 0 
msk_claims_visits_df['IP_visits_count'].fillna(0,inplace=True)
msk_claims_visits_df['OP_visits_count'].fillna(0,inplace=True)
msk_claims_visits_df.head()
msk_claims_visits_df.shape

# (8745, 6)
# No dup found!

member_id
850004357:01    0
850004357:02    1
850004965:01    2
850012059:02    3
850143894:01    4
Name: visits_count, dtype: int64

,member_id,member_location,member_age,member_gender,IP_total_paid_amount,OP_total_paid_amount
0,850004357:01,SINGAPORE,57,M,0.0,4331.169672
1,850004357:02,SINGAPORE,56,F,0.0,14455.761894
2,850004965:01,MALAYSIA,58,M,0.0,1926.120000
3,850012059:02,SINGAPORE,47,F,0.0,1332.140000
4,850143894:01,SINGAPORE,38,F,0.0,4598.690000


(9651, 6)

,member_id,member_location,member_age,member_gender,IP_visits_count,OP_visits_count
0,850004357:01,SINGAPORE,57,M,0.0,14.0
1,850004357:02,SINGAPORE,56,F,0.0,57.0
2,850004965:01,MALAYSIA,58,M,0.0,6.0
3,850012059:02,SINGAPORE,47,F,0.0,8.0
4,850143894:01,SINGAPORE,38,F,0.0,24.0


(9651, 6)

In [16]:
### Create a functionto check duplicacy ###
def duplicacy(df):
    duplicate_member_ids = df['member_id'].duplicated() 
    if duplicate_member_ids.any():
        print("Duplicate member_ids found")
        duplicate_rows = df[duplicate_member_ids]
        print(duplicate_rows)
    else:
        print("No duplicate member_ids found.")

In [17]:
### Get distinguished IP/OP visits and total_paid_amount ###
msk_claims_final=msk_claims_visits_df[['member_id','member_location','member_age','member_gender','IP_visits_count','OP_visits_count']].merge(msk_claims_amt_df[['member_id','IP_total_paid_amount','OP_total_paid_amount']],how ='inner',left_on=['member_id'], right_on=['member_id'])
msk_claims_final.head(10)
msk_claims_final.shape

# (7782, 8)

msk_claims_final=msk_claim_visits_df.merge(msk_claims_final,how ='inner',left_on=['member_id'], right_on=['member_id'])

# Renaming the market to relevant column name --- {market : market_most_visits} #
msk_claims_final.rename(columns={'market':'market_most_visits'},inplace=True)


msk_claims_final.head(10)
msk_claims_final.shape
# (7782, 11)

,member_id,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount
0,850004357:01,SINGAPORE,57,M,0.0,14.0,0.00,4331.169672
1,850004357:02,SINGAPORE,56,F,0.0,57.0,0.00,14455.761894
2,850004965:01,MALAYSIA,58,M,0.0,6.0,0.00,1926.120000
3,850012059:02,SINGAPORE,47,F,0.0,8.0,0.00,1332.140000
4,850143894:01,SINGAPORE,38,F,0.0,24.0,0.00,4598.690000
5,850167354:01,HONG KONG,61,M,0.0,22.0,0.00,9321.160000
6,850167354:02,HONG KONG,59,F,2.0,36.0,20466.53,11691.720000
7,850167355:01,HONG KONG,77,M,0.0,29.0,0.00,8040.570000
8,850174242:01,SINGAPORE,53,M,0.0,20.0,0.00,5148.890000
9,850174243:02,SINGAPORE,56,F,0.0,13.0,0.00,2382.310000


(9651, 8)

,member_id,market_most_visits,visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount
0,850004357:01,Singapore,14,8,SINGAPORE,57,M,0.0,14.0,0.00,4331.169672
1,850004357:02,Singapore,57,12,SINGAPORE,56,F,0.0,57.0,0.00,14455.761894
2,850004965:01,Hong Kong,6,5,MALAYSIA,58,M,0.0,6.0,0.00,1926.120000
3,850012059:02,Singapore,8,5,SINGAPORE,47,F,0.0,8.0,0.00,1332.140000
4,850143894:01,Singapore,24,10,SINGAPORE,38,F,0.0,24.0,0.00,4598.690000
5,850167354:01,Hong Kong,22,10,HONG KONG,61,M,0.0,22.0,0.00,9321.160000
6,850167354:02,Hong Kong,38,11,HONG KONG,59,F,2.0,36.0,20466.53,11691.720000
7,850167355:01,Hong Kong,29,5,HONG KONG,77,M,0.0,29.0,0.00,8040.570000
8,850174242:01,Hong Kong,20,9,SINGAPORE,53,M,0.0,20.0,0.00,5148.890000
9,850174243:02,Hong Kong,13,7,SINGAPORE,56,F,0.0,13.0,0.00,2382.310000


(9651, 11)

In [24]:
# Obtain unqiue members in the dataset #
msk_claims_final['member_id'].nunique() #--- # Matches with rows of the dataset!!!

9278

# Yellow flag members - data 2 years automated

In [18]:
### Create function for identifying the yellow flag members ###
query = """
SELECT member_id,diagnosis_key,incurred_date from claims 
    where market in ('Singapore','Hong Kong')     
    and incurred__date__year_month_ >=(select
                                MAX(incurred__date__year_month_)-INTERVAL '2' year
                                from "hk_i_cleansed_ghb"."claims")                             
    group by member_id,diagnosis_key,incurred_date
"""
#setting the output of the query to a dataframe df, run this prior to running the df query
yellow_members = wr.athena.read_sql_query(query,\
#specify where the query runs                             
database="hk_i_cleansed_ghb",\
boto3_session=session,\
ctas_approach=False,\
workgroup='DataScientists')
#view
yellow_members.head()

#Adjust diagnosis field
yellow_members['diagnosis_key_for_lookup_1'] = yellow_members['diagnosis_key'].str.replace('.', '')
yellow_members['diagnosis_key_for_lookup_2'] = yellow_members['diagnosis_key_for_lookup_1'].apply('{:0<5}'.format)
yellow_members['incurred_date'] = pd.to_datetime(yellow_members['incurred_date'])
yellow_members['visits_derived'] = yellow_members['member_id'].astype(str)+'_'+ yellow_members['incurred_date'].dt.date.astype(str)

yellow_members.head(3)

,member_id,diagnosis_key,incurred_date
0,850604572:05,TT,2022-01-12
1,850604576:01,V72.5,2022-01-03
2,850704209:01,TT,2022-01-26
3,850704211:02,460,2022-01-11
4,850704241:02,TT,2022-01-27


,member_id,diagnosis_key,incurred_date,diagnosis_key_for_lookup_1,diagnosis_key_for_lookup_2,visits_derived
0,850604572:05,TT,2022-01-12,TT,TT000,850604572:05_2022-01-12
1,850604576:01,V72.5,2022-01-03,V725,V7250,850604576:01_2022-01-03
2,850704209:01,TT,2022-01-26,TT,TT000,850704209:01_2022-01-26


In [19]:
def yellowFlagging_type1(df, df2, in_list, indicator_name,  indicator_agg_col):
    members_yf=df[df['diagnosis_key_for_lookup_2'].isin(in_list)].groupby(['member_id']
    ).agg(
    aggregator_col_name = ('visits_derived',pd.Series.nunique)
    ).reset_index()

    df2[indicator_name] =  df2["member_id"].isin(members_yf["member_id"]).astype(int)
    df2[indicator_name].value_counts()

    # Joining the tables to get the yellow flag visits column
    df2=df2.merge(members_yf,how ='left',on=['member_id'])

    # msk_claims_obs_overall_final.head()
    print("columns " , df2.columns)
    df2.rename(columns={'aggregator_col_name': indicator_agg_col}, inplace=True)

    print("columns " , df2.columns)
    

    df2[indicator_agg_col].fillna(0,inplace=True)
    df2[indicator_agg_col].value_counts()
    
    
    return df2


def yellowFlagging_type2(df, df2, in_list, indicator_name,  indicator_agg_col):
    members_yf=df[df['diagnosis_key_for_lookup_2'].str.startswith(tuple(in_list))].groupby(['member_id']
    ).agg(
    aggregator_col_name = ('visits_derived',pd.Series.nunique)
    ).reset_index()


    df2[indicator_name] =  df2["member_id"].isin(members_yf["member_id"]).astype(int)
    df2[indicator_name].value_counts()

    # Joining the tables to get the yellow flag visits column
    df2=df2.merge(members_yf,how ='left',on=['member_id'])
    
    print("columns " , df2.columns)
    df2.rename(columns={'aggregator_col_name': indicator_agg_col}, inplace=True)

    print("columns " , df2.columns)
    

    df2[indicator_agg_col].fillna(0,inplace=True)
    df2[indicator_agg_col].value_counts()

    
    return df2





### Create a function to compute the visit rate per 1000
def visit_rate_compute(df,visits_rate,visits_count):
    df[visits_rate]=(df[visits_count]/df['eligible_yrmth_count'])*1000
    return df



def visit_rate_compute_v2(df,visits_rate,visits_count):
    df[visits_rate]=(df[visits_count]/df['eligible_yrmth_count'])
    return df





In [20]:
msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final,['24900','24901','24960','24961','24970','24971','24980','24981','24990','24991'], 'yf249_Type1Diabetes_indicator','yf249_Type1Diabetes_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf249_Type1Diabetes_visits_rate','yf249_Type1Diabetes_visits_count')
##
msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final ,['25000','25002','25060','25062','25070','25080','25082','25090','25092'],'yf250_Type2Diabetes_indicator','yf250_Type2Diabetes_visits_count'  )
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf250_Type2Diabetes_visits_rate','yf250_Type2Diabetes_visits_count')
##
msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final , ['27200','27210','27220','27230','27230','27240','27250','27260','27270','27280','27290'],'yf272_Dyslipidemia_indicator','yf272_Dyslipidemia_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf272_Dyslipidemia_visits_rate','yf272_Dyslipidemia_visits_count')
##
msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final , ['40100','40110','40190','40200','40300','40400','40390'],'yf401_Hypertension_indicator','yf401_Hypertension_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf401_Hypertension_visits_rate','yf401_Hypertension_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , ['3051'],'yf305_TobaccoUse_indicator','yf305_TobaccoUse_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf305_TobaccoUse_visits_rate','yf305_TobaccoUse_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , ['3055'],'yf3055_illicitDrugUse_indicator','yf3055_illicitDrugUse_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf3055_illicitDrugUse_visits_count','yf3055_illicitDrugUse_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '3055','yf311_Depression_indicator','yf311_Depression_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf311_Depression_visits_rate','yf311_Depression_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '300','yf300_Anxiety_indicator','yf300_Anxiety_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf300_Anxiety_visits_rate','yf300_Anxiety_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '955','yf955_Polypharmcy_indicator','yf955_Polypharmcy_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf955_Polypharmcy_visits_rate','yf955_Polypharmcy_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '278','yf278_Obesity_indicator','yf278_Obesity_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf278_Obesity_visits_rate','yf278_Obesity_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '3002','yf3002_Kinesiophobia_indicator','yf3002_Kinesiophobia_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf3002_Kinesiophobia_visits_rate','yf3002_Kinesiophobia_visits_count')
##
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final ,['309','v60','z63'],'yf309_Acopia_indicator','yf309_Acopia_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf309_Acopia_visits_rate','yf309_Acopia_visits_count')
##

columns  Index(['member_id', 'market_most_visits', 'visits_count',
       'eligible_yrmth_count', 'member_location', 'member_age',
       'member_gender', 'IP_visits_count', 'OP_visits_count',
       'IP_total_paid_amount', 'OP_total_paid_amount',
       'yf249_Type1Diabetes_indicator', 'aggregator_col_name'],
      dtype='object')
columns  Index(['member_id', 'market_most_visits', 'visits_count',
       'eligible_yrmth_count', 'member_location', 'member_age',
       'member_gender', 'IP_visits_count', 'OP_visits_count',
       'IP_total_paid_amount', 'OP_total_paid_amount',
       'yf249_Type1Diabetes_indicator', 'yf249_Type1Diabetes_visits_count'],
      dtype='object')
columns  Index(['member_id', 'market_most_visits', 'visits_count',
       'eligible_yrmth_count', 'member_location', 'member_age',
       'member_gender', 'IP_visits_count', 'OP_visits_count',
       'IP_total_paid_amount', 'OP_total_paid_amount',
       'yf249_Type1Diabetes_indicator', 'yf249_Type1Diabetes_visits_cou

In [21]:
### Renaming and computing visit rate for respective yellow flags ###
msk_claims_final.rename(columns={'visits_count':'total_visits_count' },inplace=True)
# Adding Total gross paid amount column to the dataset
msk_claims_final["total_paid_amount"] = msk_claims_final["IP_total_paid_amount"] + msk_claims_final["OP_total_paid_amount"]

msk_claims_final = visit_rate_compute_v2(msk_claims_final,'total_visits_rate','total_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'IP_total_visits_rate','IP_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'OP_total_visits_rate','OP_visits_count')

In [22]:
### Compute PPPM value for observation period ###
msk_claims_final['pppm_obs'] = msk_claims_final['total_paid_amount']/msk_claims_final['eligible_yrmth_count']
msk_claims_final['IP_total_paid_amount_pppm'] = msk_claims_final['total_paid_amount']/msk_claims_final['eligible_yrmth_count']
msk_claims_final['OP_total_paid_amount_pppm'] = msk_claims_final['total_paid_amount']/msk_claims_final['eligible_yrmth_count']
### Cost indicator computation as well ###
msk_claims_final['cost_indicator_obs'] = msk_claims_final['pppm_obs'].apply(lambda x: int(1) if x >= 1000 else int(0))

In [33]:
msk_claims_final.head()
msk_claims_final.shape

,member_id,market_most_visits,total_visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount,yf249_Type1Diabetes_indicator,yf249_Type1Diabetes_visits_count,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_indicator,yf250_Type2Diabetes_visits_count,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_indicator,yf272_Dyslipidemia_visits_count,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_indicator,yf401_Hypertension_visits_count,yf401_Hypertension_visits_rate,yf305_TobaccoUse_indicator,yf305_TobaccoUse_visits_count,yf305_TobaccoUse_visits_rate,yf3055_illicitDrugUse_indicator,yf3055_illicitDrugUse_visits_count,yf311_Depression_indicator,yf311_Depression_visits_count,yf311_Depression_visits_rate,yf300_Anxiety_indicator,yf300_Anxiety_visits_count,yf300_Anxiety_visits_rate,yf955_Polypharmcy_indicator,yf955_Polypharmcy_visits_count,yf955_Polypharmcy_visits_rate,yf278_Obesity_indicator,yf278_Obesity_visits_count,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_indicator,yf3002_Kinesiophobia_visits_count,yf3002_Kinesiophobia_visits_rate,yf309_Acopia_indicator,yf309_Acopia_visits_count,yf309_Acopia_visits_rate,total_paid_amount,total_visits_rate,IP_total_visits_rate,OP_total_visits_rate,pppm_obs,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,cost_indicator_obs
0,850004357:01,Singapore,13,7,SINGAPORE,57,M,0.0,13.0,0.00,3960.28,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,6.0,0.857143,1,5.0,0.714286,1,1.0,0.142857,1,5.0,0.714286,1,6.0,0.857143,0,0.0,0.0,3960.28,1.857143,0.00,1.857143,565.754286,565.754286,565.754286,0
1,850004357:02,Singapore,33,10,SINGAPORE,56,F,0.0,33.0,0.00,5648.70,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,20.0,2.000000,1,18.0,1.800000,1,2.0,0.200000,1,61.0,6.100000,1,18.0,1.800000,0,0.0,0.0,5648.70,3.300000,0.00,3.300000,564.870000,564.870000,564.870000,0
2,850004965:01,Hong Kong,9,6,MALAYSIA,58,M,0.0,9.0,0.00,2979.86,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,3.0,0.500000,0,0.0,0.000000,1,5.0,0.833333,1,14.0,2.333333,1,8.0,1.333333,0,0.0,0.0,2979.86,1.500000,0.00,1.500000,496.643333,496.643333,496.643333,0
3,850009036:01,Singapore,37,8,SINGAPORE,55,M,2.0,35.0,9856.42,8416.45,0,0.0,0.0,0,0.0,0.0,1,4.0,0.5,1,1.0,0.125,0,0.0,0.0,0,0.0,1,8.0,1.000000,1,7.0,0.875000,1,3.0,0.375000,1,26.0,3.250000,1,11.0,1.375000,0,0.0,0.0,18272.87,4.625000,0.25,4.375000,2284.108750,2284.108750,2284.108750,1
4,850012059:02,Singapore,36,10,SINGAPORE,47,F,1.0,35.0,27704.71,12257.48,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,0,0.0,0.000000,0,0.0,0.000000,1,22.0,2.200000,1,17.0,1.700000,0,0.0,0.000000,0,0.0,0.0,39962.19,3.600000,0.10,3.500000,3996.219000,3996.219000,3996.219000,1


(9278, 54)

In [23]:
#### Adding metabolic syndrome indicator and identifier ####
## Adding Elevated fasting plasma glucose ###
msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final , ['79029'],'yf790_ElevatedPlasmaGlucose_indicator','yf790_ElevatedPlasmaGlucose_visits_count')
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf790_ElevatedPlasmaGlucose_visits_rate','yf790_ElevatedPlasmaGlucose_visits_count')

### Adding metabolic syndrome indetifier based on diagnosis code logic ###
msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , ['2777'],'yf277_metabolicSyndrome_identifier','yf277_metabolicSyndrome_visits_count')
### Adding visits rate for metabolic syndrome ###
msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf277_metabolicSyndrome_visits_rate','yf277_metabolicSyndrome_visits_count')

# Create a new column for metabolic syndrome indicator and initialise it with 0
msk_claims_final["yf_metabolicSyndrome_indicator"] = 0
# Check if Metabolic Identifier is True
# Count the number of True indicators
trueIndicator_count = msk_claims_final[["yf401_Hypertension_indicator","yf278_Obesity_indicator","yf790_ElevatedPlasmaGlucose_indicator","yf272_Dyslipidemia_indicator"]].sum(axis=1)

# Set the metabolic Syndrome indicator to 1 where Metabolic Indicator is True and atleast 3 other indicators are True
msk_claims_final.loc[(trueIndicator_count>=3),"yf_metabolicSyndrome_indicator"] =1 


msk_claims_final['yf_metabolicSyndrome_indicator'].value_counts()
msk_claims_final['yf277_metabolicSyndrome_identifier'].value_counts()


### Adding all yellow flag indicator column inplace ###
msk_claims_final['total_yf_indicator'] = msk_claims_final[["yf249_Type1Diabetes_indicator","yf250_Type2Diabetes_indicator","yf272_Dyslipidemia_indicator","yf401_Hypertension_indicator","yf305_TobaccoUse_indicator","yf311_Depression_indicator","yf300_Anxiety_indicator","yf955_Polypharmcy_indicator","yf309_Acopia_indicator","yf278_Obesity_indicator","yf3002_Kinesiophobia_indicator"]].max(axis=1)


columns  Index(['member_id', 'market_most_visits', 'total_visits_count',
       'eligible_yrmth_count', 'member_location', 'member_age',
       'member_gender', 'IP_visits_count', 'OP_visits_count',
       'IP_total_paid_amount', 'OP_total_paid_amount',
       'yf249_Type1Diabetes_indicator', 'yf249_Type1Diabetes_visits_count',
       'yf249_Type1Diabetes_visits_rate', 'yf250_Type2Diabetes_indicator',
       'yf250_Type2Diabetes_visits_count', 'yf250_Type2Diabetes_visits_rate',
       'yf272_Dyslipidemia_indicator', 'yf272_Dyslipidemia_visits_count',
       'yf272_Dyslipidemia_visits_rate', 'yf401_Hypertension_indicator',
       'yf401_Hypertension_visits_count', 'yf401_Hypertension_visits_rate',
       'yf305_TobaccoUse_indicator', 'yf305_TobaccoUse_visits_count',
       'yf305_TobaccoUse_visits_rate', 'yf3055_illicitDrugUse_indicator',
       'yf3055_illicitDrugUse_visits_count', 'yf311_Depression_indicator',
       'yf311_Depression_visits_count', 'yf311_Depression_visits_rate',
   

0    9436
1     215
Name: yf_metabolicSyndrome_indicator, dtype: int64

0    9619
1      32
Name: yf277_metabolicSyndrome_identifier, dtype: int64

In [24]:
#### Identifying PT procedure codes ####

def pt_procedures_type1(df, df2, procedure_str, indicator_name,indicator_agg_col):
    
    members_yf=df[df['procedure_desc'].astype(str).str.contains(procedure_str)].groupby(['member_id']
    ).agg(
    aggregator_col_name = ('visits_derived',pd.Series.nunique)
    ).reset_index()

    df2[indicator_name] =  df2["member_id"].isin(members_yf["member_id"]).astype(int)
    
    # Joining the tables to get the yellow flag visits column
    df2=df2.merge(members_yf,how ='left',on=['member_id'])
    print("columns " , df2.columns)
    df2.rename(columns={'aggregator_col_name': indicator_agg_col}, inplace=True)
    print("columns " , df2.columns)
    
    df2[indicator_agg_col].fillna(0,inplace=True)

    return df2

In [25]:
msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"PHYSO|PHYSIO|G8090S",'physiotherapy_proc_indicator','physiotherapy_visits_count')
msk_claims_final = visit_rate_compute(msk_claims_final,'physiotherapy_visits_rate','physiotherapy_visits_count')

msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final,"ARTHRO|SURG",'surgery_proc_indicator','surgery_visits_count')
msk_claims_final = visit_rate_compute(msk_claims_final,'surgery_visits_rate','surgery_visits_count')

msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"X-RAY|SCAN|MRI|ULTRASOUND|MAMMO|TOMOGRAPHY|ANGIOGRAPHY|IMAGE",'radio_diag_scans_proc_indicator','radio_diag_scans_visits_count')
msk_claims_final = visit_rate_compute(msk_claims_final,'radio_diag_scans_visits_rate','radio_diag_scans_visits_count')

msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"CONSULT|SPCAN|PHYSICIAN|PHYSICAL|PLANNING|EVAL|VIST|EXAM|SCREEN|COUNSEL|ROUTINE",'eval_manag_services_proc_indicator','eval_manag_services_visits_count')
msk_claims_final = visit_rate_compute(msk_claims_final,'eval_manag_services_visits_rate','eval_manag_services_visits_count')

msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"ALTERNATIVE|CHINESE|ACUPUNCTURE|HERBAL|MASSAGE|AYUR|OSTEOPATHY",'alternative_medicine_proc_indicator','alternative_medicine_visits_count')
msk_claims_final = visit_rate_compute(msk_claims_final,'alternative_medicine_visits_rate','alternative_medicine_visits_count')

msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"DRUG|RX|PHARMA",'drugs_related_proc_indicator','drugs_related_visits_count')
msk_claims_final = visit_rate_compute(msk_claims_final,'drugs_related_visits_rate','drugs_related_visits_count')

msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"ANAES|SEDATION",'anesthesia_proc_indicator','anesthesia_visits_count')
msk_claims_final = visit_rate_compute(msk_claims_final,'anesthesia_visits_rate','anesthesia_visits_count')


columns  Index(['member_id', 'market_most_visits', 'total_visits_count',
       'eligible_yrmth_count', 'member_location', 'member_age',
       'member_gender', 'IP_visits_count', 'OP_visits_count',
       'IP_total_paid_amount', 'OP_total_paid_amount',
       'yf249_Type1Diabetes_indicator', 'yf249_Type1Diabetes_visits_count',
       'yf249_Type1Diabetes_visits_rate', 'yf250_Type2Diabetes_indicator',
       'yf250_Type2Diabetes_visits_count', 'yf250_Type2Diabetes_visits_rate',
       'yf272_Dyslipidemia_indicator', 'yf272_Dyslipidemia_visits_count',
       'yf272_Dyslipidemia_visits_rate', 'yf401_Hypertension_indicator',
       'yf401_Hypertension_visits_count', 'yf401_Hypertension_visits_rate',
       'yf305_TobaccoUse_indicator', 'yf305_TobaccoUse_visits_count',
       'yf305_TobaccoUse_visits_rate', 'yf3055_illicitDrugUse_indicator',
       'yf3055_illicitDrugUse_visits_count', 'yf311_Depression_indicator',
       'yf311_Depression_visits_count', 'yf311_Depression_visits_rate',
   

In [26]:
#### Appending the required demographic variables in the dataset ####

# Getting the demographics variables Testing in progress
msk_elig_claim_obs['incurred_date'] = pd.to_datetime(msk_elig_claim_obs['incurred_date'])

# Sort the dataset in descending order based on incurred_date
msk_elig_claim_obs_sorted = msk_elig_claim_obs.sort_values(by="incurred_date",ascending=False)

# Create a new dataset by grouping and applying the lambda function
msk_elig_claim_obs_v2 = msk_elig_claim_obs_sorted.groupby('member_id').apply(lambda x:x.head(1).reset_index(drop=True))

msk_elig_claim_obs_subset = msk_elig_claim_obs_v2[['member_id','group_industry_sector_desc','member_family_type','member_nationality','client_location','market']]
msk_elig_claim_obs_subset.rename(columns={'market':'market_latest_index' },inplace=True)
msk_elig_claim_obs_subset.reset_index(drop=True,inplace=True)

### Merge and append the attributes for demographic variables based on latest index on visits ###
msk_claims_final = msk_claims_final.merge(msk_elig_claim_obs_subset,how ='left',on=['member_id'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
### Mapping industry ###
industry_mapping = {'Food & Textiles':'Food Textiles and Retailing',
                  'Food Retailing': 'Food Textiles and Retailing' ,
                   'Miscellaneous': 'Miscellaneous',
                  'Miscellaneous Services':'Miscellaneous' ,
                   'Other Manufacturing':'Manufacturing and Retail',
                   'Other Retail': 'Manufacturing and Retail',
                   'Passenger Transportation':'Transportation and Services',
                   'Transport Services':'Transportation and Services'}

def map_to_new_category(old_industry):
    return industry_mapping.get(old_industry,old_industry)

msk_claims_final['member_industry'] = msk_claims_final['group_industry_sector_desc'].apply(map_to_new_category)
msk_claims_final['member_industry'].value_counts()


Miscellaneous                                         3744
Banking,Investment & Insurance (inc.Brokers,Agent)    2326
Communications & Computing                            1351
Consultancy, Marketing & Legal services                944
Education                                              542
Hotels & Restaurants                                   291
Mining & Quarrying (incl. Oil & Gas Extraction)        167
Electricity & Gas Supply, Water, Sewage & Drainage      82
Wholesale                                               47
Health                                                  46
Transportation and Services                             36
Manufacturing and Retail                                31
Food Textiles and Retailing                             23
Dev, Buying & Selling of Domestic or Comm.Property      11
Chemicals, Petroleum Refining & Assoc.Products           4
Light, Electrical & Precision Manufacturing              2
Freight Transportation & Warehousing                    

# Mapping countries to respective regions

In [28]:
### Mapping countries to respective regions ###
df_country_region = pd.read_csv('s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/countryISOreference.csv',encoding='ISO-8859-1')
country_to_region_dict = dict(zip(df_country_region['countryShortNameISO'],df_country_region['Region']))

capitalize_country_to_region_dict = {(k.upper() if isinstance(k,str) else k):
                                     (v.upper() if isinstance (v,str) else v)
                                     for k,v in country_to_region_dict.items()}

country_to_region_dict = capitalize_country_to_region_dict


In [40]:
country_to_region_dict

{'ANTARCTICA': nan,
 'ANGOLA': 'AFRICA',
 'BURKINA FASO': 'AFRICA',
 'BURUNDI': 'AFRICA',
 'BENIN': 'AFRICA',
 'BOTSWANA': 'AFRICA',
 'CONGO (THE DEMOCRATIC REPUBLIC OF THE)': 'AFRICA',
 'CENTRAL AFRICAN REPUBLIC (THE)': 'AFRICA',
 'CONGO (THE)': 'AFRICA',
 "CÔTE D'IVOIRE": 'AFRICA',
 'CAMEROON': 'AFRICA',
 'CABO VERDE': 'AFRICA',
 'DJIBOUTI': 'AFRICA',
 'ALGERIA': 'AFRICA',
 'WESTERN SAHARA*': 'AFRICA',
 'ERITREA': 'AFRICA',
 'ETHIOPIA': 'AFRICA',
 'GABON': 'AFRICA',
 'GHANA': 'AFRICA',
 'GAMBIA (THE)': 'AFRICA',
 'GUINEA': 'AFRICA',
 'EQUATORIAL GUINEA': 'AFRICA',
 'GUINEA-BISSAU': 'AFRICA',
 'KENYA': 'AFRICA',
 'COMOROS (THE)': 'AFRICA',
 'LIBERIA': 'AFRICA',
 'LESOTHO': 'AFRICA',
 'LIBYA': 'AFRICA',
 'MOROCCO': 'AFRICA',
 'MADAGASCAR': 'AFRICA',
 'MALI': 'AFRICA',
 'MAURITANIA': 'AFRICA',
 'MAURITIUS': 'AFRICA',
 'MALAWI': 'AFRICA',
 'MOZAMBIQUE': 'AFRICA',
 'NAMIBIA': 'AFRICA',
 'NIGER (THE)': 'AFRICA',
 'NIGERIA': 'AFRICA',
 'RÉUNION': 'AFRICA',
 'RWANDA': 'AFRICA',
 'SEYCHELLES'

## fuzzy wuzzy for location

In [29]:
!pip install fuzzywuzzy

from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

def find_best_match(member_location):
    best_match, score = process.extractOne(member_location,country_to_region_dict.keys())
    
    if score > 80 :
        return country_to_region_dict[best_match]
    else:
        return member_location

msk_claims_final['member_location_region'] = msk_claims_final['member_location'].apply(find_best_match)
msk_claims_final['member_nationality_region'] = msk_claims_final['member_nationality'].apply(find_best_match)

## Adding Maternity indicator

In [30]:
##### Adding maternity indicator to the dataset

def maternity_indicator(df,column_name,in_list,indicator_name,df2):
    mat_mem = df[df[column_name].isin(in_list)].groupby(['member_id']
    ).agg(
    members_distinct_count=('member_id', pd.Series.nunique),
    ).reset_index()

    df2[indicator_name] =  df2["member_id"].isin(mat_mem["member_id"]).astype(int)

    df2[indicator_name].value_counts()
    return df2

msk_claims_final = maternity_indicator(msk_elig_claim_obs,'procedure_code',['R1820S','R1820A','R2490A','R2290S','BPUMP','MATGS'],'maternity_procedure_indicator',msk_claims_final)
msk_claims_final = maternity_indicator(msk_elig_claim_obs,'case_type_0001',['CM','NM'],'maternity_case_type_indicator',msk_claims_final)
#### Create a indicator for maternity members with diagnosis code ####
mat_codes = [i for i in range(630,780)]
mat_codes = [str(x*100) for x in mat_codes]

msk_claims_final = maternity_indicator(msk_elig_claim_obs,'diagnosis_key_for_lookup_2',mat_codes,'maternity_diagnosis_indicator',msk_claims_final)

#### Create all_maternity_indicator column ####
msk_claims_final['all_maternity_indicator'] = msk_claims_final[["maternity_procedure_indicator","maternity_case_type_indicator","maternity_diagnosis_indicator"]].max(axis=1)



In [45]:
msk_claims_final.head()

,member_id,market_most_visits,total_visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount,yf249_Type1Diabetes_indicator,yf249_Type1Diabetes_visits_count,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_indicator,yf250_Type2Diabetes_visits_count,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_indicator,yf272_Dyslipidemia_visits_count,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_indicator,yf401_Hypertension_visits_count,yf401_Hypertension_visits_rate,yf305_TobaccoUse_indicator,yf305_TobaccoUse_visits_count,yf305_TobaccoUse_visits_rate,yf3055_illicitDrugUse_indicator,yf3055_illicitDrugUse_visits_count,yf311_Depression_indicator,yf311_Depression_visits_count,yf311_Depression_visits_rate,yf300_Anxiety_indicator,yf300_Anxiety_visits_count,yf300_Anxiety_visits_rate,yf955_Polypharmcy_indicator,yf955_Polypharmcy_visits_count,yf955_Polypharmcy_visits_rate,yf278_Obesity_indicator,yf278_Obesity_visits_count,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_indicator,yf3002_Kinesiophobia_visits_count,yf3002_Kinesiophobia_visits_rate,yf309_Acopia_indicator,yf309_Acopia_visits_count,yf309_Acopia_visits_rate,total_paid_amount,total_visits_rate,IP_total_visits_rate,OP_total_visits_rate,pppm_obs,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,cost_indicator_obs,yf790_ElevatedPlasmaGlucose_indicator,yf790_ElevatedPlasmaGlucose_visits_count,yf790_ElevatedPlasmaGlucose_visits_rate,yf277_metabolicSyndrome_identifier,yf277_metabolicSyndrome_visits_count,yf277_metabolicSyndrome_visits_rate,yf_metabolicSyndrome_indicator,total_yf_indicator,physiotherapy_proc_indicator,physiotherapy_visits_count,physiotherapy_visits_rate,surgery_proc_indicator,surgery_visits_count,surgery_visits_rate,radio_diag_scans_proc_indicator,radio_diag_scans_visits_count,radio_diag_scans_visits_rate,eval_manag_services_proc_indicator,eval_manag_services_visits_count,eval_manag_services_visits_rate,alternative_medicine_proc_indicator,alternative_medicine_visits_count,alternative_medicine_visits_rate,drugs_related_proc_indicator,drugs_related_visits_count,drugs_related_visits_rate,anesthesia_proc_indicator,anesthesia_visits_count,anesthesia_visits_rate,group_industry_sector_desc,member_family_type,member_nationality,client_location,market_latest_index,member_industry,member_location_region,member_nationality_region,maternity_procedure_indicator,maternity_case_type_indicator,maternity_diagnosis_indicator,all_maternity_indicator
0,850004357:01,Singapore,13,7,SINGAPORE,57,M,0.0,13.0,0.00,3960.28,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,6.0,0.857143,1,5.0,0.714286,1,1.0,0.142857,1,5.0,0.714286,1,6.0,0.857143,0,0.0,0.0,3960.28,1.857143,0.00,1.857143,565.754286,565.754286,565.754286,0,0,0.0,0.0,0,0.0,0.0,0,1,0,0.0,0.0,0,0.0,0.0,1,1.0,142.857143,1,7.0,1000.0,1,2.0,285.714286,1,5.0,714.285714,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,Singapore,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0
1,850004357:02,Singapore,33,10,SINGAPORE,56,F,0.0,33.0,0.00,5648.70,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,20.0,2.000000,1,18.0,1.800000,1,2.0,0.200000,1,61.0,6.100000,1,18.0,1.800000,0,0.0,0.0,5648.70,3.300000,0.00,3.300000,564.870000,564.870000,564.870000,0,0,0.0,0.0,0,0.0,0.0,0,1,1,10.0,1000.0,0,0.0,0.0,1,1.0,100.000000,1,12.0,1200.0,1,5.0,500.000000,1,6.0,600.000000,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,Singapore,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0
2,850004965:01,Hong Kong,9,6,MALAYSIA,58,M,0.0,9.0,0.00,2979.86,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,3.0,0.500000,0,0.0,0.000000,1,5.0,0.833333,1,14.0,2.333333,1,8.0,1.333333,0,0.0,0.0,2979.86,1.500000,0.00,1.500000,496.643333,496.643333,496.643333,0,0,0.0,0.0,0,0.0,0.0,0,1,0,0.0,0.0,0,0.0,0.0,1,1.0,166.666667,1,6.0,1000.0,0,0.0,0.000000,

# Extracting peformance period of three months-date automated

In [35]:
msk_elig_claims_perf=msk_elig_claim.loc[(msk_elig_claim['incurred__date__year_month_']>=(msk_elig_claim['incurred__date__year_month_'].max()-pd.DateOffset(months=3)
                                                                                        +pd.DateOffset(days=1)))
                   & (msk_elig_claim['incurred__date__year_month_']<=(msk_elig_claim['incurred__date__year_month_'].max())) ] 

In [36]:
per_start_date=msk_elig_claim['incurred__date__year_month_'].max()-pd.DateOffset(months=3)+pd.DateOffset(days=1)
per_end_date=msk_elig_claim['incurred__date__year_month_'].max()

print(per_start_date)
print(per_end_date)

2023-09-29 00:00:00
2023-12-28 00:00:00


In [37]:
wr.s3.to_csv(
    df=msk_elig_claims_perf,
    path='s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/PERFORMANCE.csv') 

{'paths': ['s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/PERFORMANCE.csv'],
 'partitions_values': {}}

In [48]:
#### Appending performance period columns

#### Extracting performance period of 3 months in 2022 ####

# msk_elig_claims_perf = msk_elig_claim.loc[(msk_elig_claim['incurred_yrmth'] >= 202301) & (msk_elig_claim['incurred_yrmth'] <= 202303)]
msk_elig_claims_perf.shape

print("Distinct Member claims in 12 month period:",msk_elig_claims_perf['member_id'].nunique())

elg_df_perf = msk_elig_claims_perf.groupby(['member_id']).agg(
    eligible_yrmth_count_perf=('eligible_yrmth', 'nunique'),
    total_paid_amount_perf=('gross_paid_amount', 'sum')
)
elg_df_perf = elg_df_perf.reset_index()
elg_df_perf.head()
elg_df_perf.shape

elg_df_perf['pppm_perf'] = elg_df_perf['total_paid_amount_perf']/elg_df_perf['eligible_yrmth_count_perf']
elg_df_perf['cost_indicator_perf'] = elg_df_perf['pppm_perf'].apply(lambda x: int(1) if x >= 1000 else int(0))

msk_claims_final =msk_claims_final.merge(elg_df_perf,how ='left',on=['member_id'])

(121774, 113)

Distinct Member claims in 12 month period: 8163


,member_id,eligible_yrmth_count_perf,total_paid_amount_perf
0,850004357:01,2,1589.597312
1,850004357:02,3,4996.664838
2,850004965:01,1,401.880000
3,850012059:02,2,336.250000
4,850143894:01,4,650.860000


(8163, 3)

In [49]:
msk_claims_final.shape


(9278, 99)

In [50]:
# msk_claims_final['member_family_type_C'].value_counts() 

KeyError: 'member_family_type_C'

In [51]:
# Manipulate the suggested columns to missing --- 04-12-2023
values_to_replace = {'General Construction & Related Services':pd.NA}
msk_claims_final['member_industry'] = msk_claims_final['member_industry'].replace(values_to_replace)

In [52]:
msk_claims_final['member_industry'].value_counts()

Miscellaneous                                         3673
Banking,Investment & Insurance (inc.Brokers,Agent)    2053
Communications & Computing                            1367
Consultancy, Marketing & Legal services                886
Education                                              558
Hotels & Restaurants                                   288
Mining & Quarrying (incl. Oil & Gas Extraction)        165
Electricity & Gas Supply, Water, Sewage & Drainage      81
Wholesale                                               49
Health                                                  45
Transportation and Services                             41
Food Textiles and Retailing                             24
Manufacturing and Retail                                23
Dev, Buying & Selling of Domestic or Comm.Property      12
Chemicals, Petroleum Refining & Assoc.Products           6
Freight Transportation & Warehousing                     2
Light, Electrical & Precision Manufacturing             

In [53]:
# Manipulate the suggested columns to missing --- 04-12-2023----- (market_latest_index)
values_to_replace = {'Singapore':'market1','Hong Kong':'market2'}
msk_claims_final['market_latest_index'] = msk_claims_final['market_latest_index'].replace(values_to_replace)

In [54]:
msk_claims_final['market_latest_index'].value_counts()

market1    6849
market2    2429
Name: market_latest_index, dtype: int64

In [55]:
msk_claims_final.head()
# member_nationality
msk_claims_final['member_nationality'].value_counts()

,member_id,market_most_visits,total_visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount,yf249_Type1Diabetes_indicator,yf249_Type1Diabetes_visits_count,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_indicator,yf250_Type2Diabetes_visits_count,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_indicator,yf272_Dyslipidemia_visits_count,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_indicator,yf401_Hypertension_visits_count,yf401_Hypertension_visits_rate,yf305_TobaccoUse_indicator,yf305_TobaccoUse_visits_count,yf305_TobaccoUse_visits_rate,yf3055_illicitDrugUse_indicator,yf3055_illicitDrugUse_visits_count,yf311_Depression_indicator,yf311_Depression_visits_count,yf311_Depression_visits_rate,yf300_Anxiety_indicator,yf300_Anxiety_visits_count,yf300_Anxiety_visits_rate,yf955_Polypharmcy_indicator,yf955_Polypharmcy_visits_count,yf955_Polypharmcy_visits_rate,yf278_Obesity_indicator,yf278_Obesity_visits_count,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_indicator,yf3002_Kinesiophobia_visits_count,yf3002_Kinesiophobia_visits_rate,yf309_Acopia_indicator,yf309_Acopia_visits_count,yf309_Acopia_visits_rate,total_paid_amount,total_visits_rate,IP_total_visits_rate,OP_total_visits_rate,pppm_obs,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,cost_indicator_obs,yf790_ElevatedPlasmaGlucose_indicator,yf790_ElevatedPlasmaGlucose_visits_count,yf790_ElevatedPlasmaGlucose_visits_rate,yf277_metabolicSyndrome_identifier,yf277_metabolicSyndrome_visits_count,yf277_metabolicSyndrome_visits_rate,yf_metabolicSyndrome_indicator,total_yf_indicator,physiotherapy_proc_indicator,physiotherapy_visits_count,physiotherapy_visits_rate,surgery_proc_indicator,surgery_visits_count,surgery_visits_rate,radio_diag_scans_proc_indicator,radio_diag_scans_visits_count,radio_diag_scans_visits_rate,eval_manag_services_proc_indicator,eval_manag_services_visits_count,eval_manag_services_visits_rate,alternative_medicine_proc_indicator,alternative_medicine_visits_count,alternative_medicine_visits_rate,drugs_related_proc_indicator,drugs_related_visits_count,drugs_related_visits_rate,anesthesia_proc_indicator,anesthesia_visits_count,anesthesia_visits_rate,group_industry_sector_desc,member_family_type,member_nationality,client_location,market_latest_index,member_industry,member_location_region,member_nationality_region,maternity_procedure_indicator,maternity_case_type_indicator,maternity_diagnosis_indicator,all_maternity_indicator,eligible_yrmth_count_perf,total_paid_amount_perf,pppm_perf,cost_indicator_perf
0,850004357:01,Singapore,13,7,SINGAPORE,57,M,0.0,13.0,0.00,3960.28,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,6.0,0.857143,1,5.0,0.714286,1,1.0,0.142857,1,5.0,0.714286,1,6.0,0.857143,0,0.0,0.0,3960.28,1.857143,0.00,1.857143,565.754286,565.754286,565.754286,0,0,0.0,0.0,0,0.0,0.0,0,1,0,0.0,0.0,0,0.0,0.0,1,1.0,142.857143,1,7.0,1000.0,1,2.0,285.714286,1,5.0,714.285714,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,market1,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0,2.0,1589.597312,794.798656,0.0
1,850004357:02,Singapore,33,10,SINGAPORE,56,F,0.0,33.0,0.00,5648.70,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,20.0,2.000000,1,18.0,1.800000,1,2.0,0.200000,1,61.0,6.100000,1,18.0,1.800000,0,0.0,0.0,5648.70,3.300000,0.00,3.300000,564.870000,564.870000,564.870000,0,0,0.0,0.0,0,0.0,0.0,0,1,1,10.0,1000.0,0,0.0,0.0,1,1.0,100.000000,1,12.0,1200.0,1,5.0,500.000000,1,6.0,600.000000,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,market1,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0,3.0,4996.664838,1665.554946,1.0
2,850004965:01,Hong Kong,9,6,MALAYSIA,58,M,0.0,9.0,0.00,2979.86,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,3.0,0.500000,0,0.0,0.000000,1,5.0,0.833333,1,14.0,2.333333,1,8.0,1.333333,0,0.0,0.0,2979.86,1.500000,

SINGAPORE               3977
HONG KONG                970
CHINA                    744
INDIA                    547
UNITED KINGDOM           457
UNITED STATES            419
MALAYSIA                 338
AUSTRALIA                294
SOUTH KOREA              181
INDONESIA                153
CANADA                   152
PHILIPPINES              135
FRANCE                   100
JAPAN                     92
TAIWAN                    89
NEW ZEALAND               74
THAILAND                  66
VIET NAM                  50
GERMANY                   43
RUSSIAN FEDERATION        27
ITALY                     27
IRELAND                   24
NETHERLANDS               24
SOUTH AFRICA              21
SWEDEN                    21
SPAIN                     20
SWITZERLAND               19
BELGIUM                   17
PAKISTAN                  15
SRI LANKA                 15
BRAZIL                    14
DENMARK                   10
PORTUGAL                  10
POLAND                     8
MYANMAR       

In [56]:
# Delete US nationals from the dataset
msk_claims_final_v2 = msk_claims_final.drop(msk_claims_final[msk_claims_final['member_nationality'] == 'UNITED STATES'].index)

In [57]:
msk_claims_final_v2.shape

(8859, 99)

In [58]:
msk_claims_final_v2['member_nationality'].value_counts()

SINGAPORE               3977
HONG KONG                970
CHINA                    744
INDIA                    547
UNITED KINGDOM           457
MALAYSIA                 338
AUSTRALIA                294
SOUTH KOREA              181
INDONESIA                153
CANADA                   152
PHILIPPINES              135
FRANCE                   100
JAPAN                     92
TAIWAN                    89
NEW ZEALAND               74
THAILAND                  66
VIET NAM                  50
GERMANY                   43
ITALY                     27
RUSSIAN FEDERATION        27
NETHERLANDS               24
IRELAND                   24
SWEDEN                    21
SOUTH AFRICA              21
SPAIN                     20
SWITZERLAND               19
BELGIUM                   17
PAKISTAN                  15
SRI LANKA                 15
BRAZIL                    14
DENMARK                   10
PORTUGAL                  10
POLAND                     8
MYANMAR                    7
NORWAY        

In [61]:
msk_claims_final_v2.dtypes

member_id                                    object
market_most_visits                           object
total_visits_count                            int64
eligible_yrmth_count                          int64
member_location                              object
member_age                                    int64
member_gender                                object
IP_visits_count                             float64
OP_visits_count                             float64
IP_total_paid_amount                        float64
OP_total_paid_amount                        float64
yf249_Type1Diabetes_indicator                 int64
yf249_Type1Diabetes_visits_count            float64
yf249_Type1Diabetes_visits_rate             float64
yf250_Type2Diabetes_indicator                 int64
yf250_Type2Diabetes_visits_count            float64
yf250_Type2Diabetes_visits_rate             float64
yf272_Dyslipidemia_indicator                  int64
yf272_Dyslipidemia_visits_count             float64
yf272_Dyslip

In [62]:
# Manipulate the suggested columns to missing --- 04-12-2023
values_to_replace = {'General Construction & Related Services':pd.NA}
msk_claims_final['member_industry'] = msk_claims_final['member_industry'].replace(values_to_replace)

In [63]:
msk_claims_final_v2.head()

,member_id,market_most_visits,total_visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount,yf249_Type1Diabetes_indicator,yf249_Type1Diabetes_visits_count,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_indicator,yf250_Type2Diabetes_visits_count,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_indicator,yf272_Dyslipidemia_visits_count,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_indicator,yf401_Hypertension_visits_count,yf401_Hypertension_visits_rate,yf305_TobaccoUse_indicator,yf305_TobaccoUse_visits_count,yf305_TobaccoUse_visits_rate,yf3055_illicitDrugUse_indicator,yf3055_illicitDrugUse_visits_count,yf311_Depression_indicator,yf311_Depression_visits_count,yf311_Depression_visits_rate,yf300_Anxiety_indicator,yf300_Anxiety_visits_count,yf300_Anxiety_visits_rate,yf955_Polypharmcy_indicator,yf955_Polypharmcy_visits_count,yf955_Polypharmcy_visits_rate,yf278_Obesity_indicator,yf278_Obesity_visits_count,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_indicator,yf3002_Kinesiophobia_visits_count,yf3002_Kinesiophobia_visits_rate,yf309_Acopia_indicator,yf309_Acopia_visits_count,yf309_Acopia_visits_rate,total_paid_amount,total_visits_rate,IP_total_visits_rate,OP_total_visits_rate,pppm_obs,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,cost_indicator_obs,yf790_ElevatedPlasmaGlucose_indicator,yf790_ElevatedPlasmaGlucose_visits_count,yf790_ElevatedPlasmaGlucose_visits_rate,yf277_metabolicSyndrome_identifier,yf277_metabolicSyndrome_visits_count,yf277_metabolicSyndrome_visits_rate,yf_metabolicSyndrome_indicator,total_yf_indicator,physiotherapy_proc_indicator,physiotherapy_visits_count,physiotherapy_visits_rate,surgery_proc_indicator,surgery_visits_count,surgery_visits_rate,radio_diag_scans_proc_indicator,radio_diag_scans_visits_count,radio_diag_scans_visits_rate,eval_manag_services_proc_indicator,eval_manag_services_visits_count,eval_manag_services_visits_rate,alternative_medicine_proc_indicator,alternative_medicine_visits_count,alternative_medicine_visits_rate,drugs_related_proc_indicator,drugs_related_visits_count,drugs_related_visits_rate,anesthesia_proc_indicator,anesthesia_visits_count,anesthesia_visits_rate,group_industry_sector_desc,member_family_type,member_nationality,client_location,market_latest_index,member_industry,member_location_region,member_nationality_region,maternity_procedure_indicator,maternity_case_type_indicator,maternity_diagnosis_indicator,all_maternity_indicator,eligible_yrmth_count_perf,total_paid_amount_perf,pppm_perf,cost_indicator_perf
0,850004357:01,Singapore,13,7,SINGAPORE,57,M,0.0,13.0,0.00,3960.28,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,6.0,0.857143,1,5.0,0.714286,1,1.0,0.142857,1,5.0,0.714286,1,6.0,0.857143,0,0.0,0.0,3960.28,1.857143,0.00,1.857143,565.754286,565.754286,565.754286,0,0,0.0,0.0,0,0.0,0.0,0,1,0,0.0,0.0,0,0.0,0.0,1,1.0,142.857143,1,7.0,1000.0,1,2.0,285.714286,1,5.0,714.285714,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,market1,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0,2.0,1589.597312,794.798656,0.0
1,850004357:02,Singapore,33,10,SINGAPORE,56,F,0.0,33.0,0.00,5648.70,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,20.0,2.000000,1,18.0,1.800000,1,2.0,0.200000,1,61.0,6.100000,1,18.0,1.800000,0,0.0,0.0,5648.70,3.300000,0.00,3.300000,564.870000,564.870000,564.870000,0,0,0.0,0.0,0,0.0,0.0,0,1,1,10.0,1000.0,0,0.0,0.0,1,1.0,100.000000,1,12.0,1200.0,1,5.0,500.000000,1,6.0,600.000000,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,market1,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0,3.0,4996.664838,1665.554946,1.0
2,850004965:01,Hong Kong,9,6,MALAYSIA,58,M,0.0,9.0,0.00,2979.86,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,3.0,0.500000,0,0.0,0.000000,1,5.0,0.833333,1,14.0,2.333333,1,8.0,1.333333,0,0.0,0.0,2979.86,1.500000,

In [64]:
msk_claims_final_v2['member_family_type'].value_counts()

F    4437
S    2126
C    2100
D     186
#      10
Name: member_family_type, dtype: Int64

In [65]:
# Manipulate the suggested columns to missing --- 04-12-2023----member_family_type
values_to_replace = {'#':pd.NA}
msk_claims_final_v2['member_family_type'] = msk_claims_final_v2['member_family_type'].replace(values_to_replace)

In [66]:
msk_claims_final_v2['member_family_type'].value_counts()

F    4437
S    2126
C    2100
D     186
Name: member_family_type, dtype: int64

In [67]:
df=msk_claims_final_v2

# Why sending to s3

In [71]:
# df[df["member_family_type"].isin(["#"])]
# df.groupby(['member_family_type']).agg(count1=('member_id', 'count'))

In [72]:
# df.groupby(['market_latest_index']).agg(count1=('member_id', 'count'))

In [73]:
df2 = msk_claims_final_v2.rename(columns={'physiotherapy_visits_rate': 'physiotherapy_visits_rate',
#                          'surgery_visits_rate': 'surgery_proc_visits_rate',
                          'surgery_proc_visits_rate': 'surgery_visits_rate',
#                          'radio_diag_scans_visits_rate': 'radio_diag_scans_proc_visits_rate',
                         'radio_diag_scans_proc_visits_rate': 'radio_diag_scans_visits_rate',
#                          'eval_manag_services_visits_rate': 'eval_manag_services_proc_visits_rate',
                         'eval_manag_services_proc_visits_rate': 'eval_manag_services_visits_rate',
#                          'alternative_medicine_visits_rate': 'alternative_medicine_visits_rate',
#                          'drugs_related_visits_rate': 'drugs_related_visits_rate',
#                          'anesthesia_visits_rate':'anesthesia_visits_rate',
#                          'cost_indicator_2021':'cost_indicator_obs',  
#                          'new_group_industry':'member_industry'
                        })

df2.head()

,member_id,market_most_visits,total_visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount,yf249_Type1Diabetes_indicator,yf249_Type1Diabetes_visits_count,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_indicator,yf250_Type2Diabetes_visits_count,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_indicator,yf272_Dyslipidemia_visits_count,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_indicator,yf401_Hypertension_visits_count,yf401_Hypertension_visits_rate,yf305_TobaccoUse_indicator,yf305_TobaccoUse_visits_count,yf305_TobaccoUse_visits_rate,yf3055_illicitDrugUse_indicator,yf3055_illicitDrugUse_visits_count,yf311_Depression_indicator,yf311_Depression_visits_count,yf311_Depression_visits_rate,yf300_Anxiety_indicator,yf300_Anxiety_visits_count,yf300_Anxiety_visits_rate,yf955_Polypharmcy_indicator,yf955_Polypharmcy_visits_count,yf955_Polypharmcy_visits_rate,yf278_Obesity_indicator,yf278_Obesity_visits_count,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_indicator,yf3002_Kinesiophobia_visits_count,yf3002_Kinesiophobia_visits_rate,yf309_Acopia_indicator,yf309_Acopia_visits_count,yf309_Acopia_visits_rate,total_paid_amount,total_visits_rate,IP_total_visits_rate,OP_total_visits_rate,pppm_obs,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,cost_indicator_obs,yf790_ElevatedPlasmaGlucose_indicator,yf790_ElevatedPlasmaGlucose_visits_count,yf790_ElevatedPlasmaGlucose_visits_rate,yf277_metabolicSyndrome_identifier,yf277_metabolicSyndrome_visits_count,yf277_metabolicSyndrome_visits_rate,yf_metabolicSyndrome_indicator,total_yf_indicator,physiotherapy_proc_indicator,physiotherapy_visits_count,physiotherapy_visits_rate,surgery_proc_indicator,surgery_visits_count,surgery_visits_rate,radio_diag_scans_proc_indicator,radio_diag_scans_visits_count,radio_diag_scans_visits_rate,eval_manag_services_proc_indicator,eval_manag_services_visits_count,eval_manag_services_visits_rate,alternative_medicine_proc_indicator,alternative_medicine_visits_count,alternative_medicine_visits_rate,drugs_related_proc_indicator,drugs_related_visits_count,drugs_related_visits_rate,anesthesia_proc_indicator,anesthesia_visits_count,anesthesia_visits_rate,group_industry_sector_desc,member_family_type,member_nationality,client_location,market_latest_index,member_industry,member_location_region,member_nationality_region,maternity_procedure_indicator,maternity_case_type_indicator,maternity_diagnosis_indicator,all_maternity_indicator,eligible_yrmth_count_perf,total_paid_amount_perf,pppm_perf,cost_indicator_perf
0,850004357:01,Singapore,13,7,SINGAPORE,57,M,0.0,13.0,0.00,3960.28,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,6.0,0.857143,1,5.0,0.714286,1,1.0,0.142857,1,5.0,0.714286,1,6.0,0.857143,0,0.0,0.0,3960.28,1.857143,0.00,1.857143,565.754286,565.754286,565.754286,0,0,0.0,0.0,0,0.0,0.0,0,1,0,0.0,0.0,0,0.0,0.0,1,1.0,142.857143,1,7.0,1000.0,1,2.0,285.714286,1,5.0,714.285714,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,market1,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0,2.0,1589.597312,794.798656,0.0
1,850004357:02,Singapore,33,10,SINGAPORE,56,F,0.0,33.0,0.00,5648.70,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,20.0,2.000000,1,18.0,1.800000,1,2.0,0.200000,1,61.0,6.100000,1,18.0,1.800000,0,0.0,0.0,5648.70,3.300000,0.00,3.300000,564.870000,564.870000,564.870000,0,0,0.0,0.0,0,0.0,0.0,0,1,1,10.0,1000.0,0,0.0,0.0,1,1.0,100.000000,1,12.0,1200.0,1,5.0,500.000000,1,6.0,600.000000,0,0.0,0.0,"Banking,Investment & Insurance (inc.Brokers,Ag...",F,SWITZERLAND,Singapore,market1,"Banking,Investment & Insurance (inc.Brokers,Ag...",ASIA,EUROPE,0,0,0,0,3.0,4996.664838,1665.554946,1.0
2,850004965:01,Hong Kong,9,6,MALAYSIA,58,M,0.0,9.0,0.00,2979.86,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,3.0,0.500000,0,0.0,0.000000,1,5.0,0.833333,1,14.0,2.333333,1,8.0,1.333333,0,0.0,0.0,2979.86,1.500000,

In [74]:
# #one-hot encoding 
# import pandas as pd
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [75]:
# df3=pd.get_dummies(df2, columns=[
    
# #'all_maternity_indicator',

# 'member_gender',
# 'member_family_type',
    
# 'market_latest_index',
# 'member_industry',
    
# 'member_location_region', 

# ], drop_first=False).drop_duplicates()

# # df3 = df3.rename(columns = lambda x:re.sub('[^A-Za-z0-9_#&]+', '_',x))
# df3.head()

In [78]:
df2.dtypes

member_id                                    object
market_most_visits                           object
total_visits_count                            int64
eligible_yrmth_count                          int64
member_location                              object
member_age                                    int64
member_gender                                object
IP_visits_count                             float64
OP_visits_count                             float64
IP_total_paid_amount                        float64
OP_total_paid_amount                        float64
yf249_Type1Diabetes_indicator                 int64
yf249_Type1Diabetes_visits_count            float64
yf249_Type1Diabetes_visits_rate             float64
yf250_Type2Diabetes_indicator                 int64
yf250_Type2Diabetes_visits_count            float64
yf250_Type2Diabetes_visits_rate             float64
yf272_Dyslipidemia_indicator                  int64
yf272_Dyslipidemia_visits_count             float64
yf272_Dyslip

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8859 entries, 0 to 9277
Data columns (total 99 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   member_id                                 8859 non-null   object 
 1   market_most_visits                        8859 non-null   object 
 2   total_visits_count                        8859 non-null   int64  
 3   eligible_yrmth_count                      8859 non-null   int64  
 4   member_location                           8859 non-null   object 
 5   member_age                                8859 non-null   int64  
 6   member_gender                             8859 non-null   object 
 7   IP_visits_count                           8859 non-null   float64
 8   OP_visits_count                           8859 non-null   float64
 9   IP_total_paid_amount                      8859 non-null   float64
 10  OP_total_paid_amount                

In [104]:
cols_to_fill=['member_gender','member_family_type','market_latest_index','member_industry','member_location_region']
df[cols_to_fill]=df[cols_to_fill].fillna('')
df2[cols_to_fill]=df2[cols_to_fill].fillna('')



In [109]:
#manual encoding
# 'member_gender' 
####
def member_gender_F(x):
    if x['member_gender'] in['F']:return 1
    else: return 0 
df2['member_gender_F']=df.apply(member_gender_F, axis=1)

df2['member_gender_F'] =df2['member_gender_F'].astype('float64')
#####
def member_gender_M(x):
    if x['member_gender'] in['M']:return 1
    else: return 0 
df2['member_gender_M']=df.apply(member_gender_M, axis=1)


# 'member_family_type'
####
def member_family_type_C(x):
    if x['member_family_type'] in['C']:return 1
    else: return 0 
df2['member_family_type_C']=df.apply(member_family_type_C, axis=1)
#####
def member_family_type_D(x):
    if x['member_family_type'] in['D']:return 1
    else: return 0 
df2['member_family_type_D']=df.apply(member_family_type_D, axis=1)
#####
def member_family_type_F(x):
    if x['member_family_type'] in['F']:return 1
    else: return 0 
df2['member_family_type_F']=df.apply(member_family_type_F, axis=1)

#####
def member_family_type_S(x):
    if x['member_family_type'] in['S']:return 1
    else: return 0 
df2['member_family_type_S']=df.apply(member_family_type_S, axis=1)


# 'market_latest_index'
#####
def market_latest_index_market1(x):
    if x['market_latest_index'] in['market1']:return 1
    else: return 0 
df2['market_latest_index_market1']=df.apply(market_latest_index_market1, axis=1)

#####
def market_latest_index_market2(x):
    if x['market_latest_index'] in['market2']:return 1
    else: return 0 
df2['market_latest_index_market2']=df.apply(market_latest_index_market2, axis=1)


# 'member_industry'
####
def member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_(x):
    if x['member_industry'] in['Banking,Investment & Insurance (inc.Brokers,Agent)']:return 1
    else: return 0 
# setattr(sys.modules[__name__], 'member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_', member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_)
df2['member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_']=df.apply(member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_, axis=1)
df2.rename(columns={'member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_':'member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_'}, inplace=True)
#####
def member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products(x):
    if x['member_industry'] in['Chemicals, Petroleum Refining & Assoc.Products']:return 1
    else: return 0 
df2['member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products']=df.apply(member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products, axis=1)
df2.rename(columns={'member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products':'member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products'}, inplace=True)
#####
def member_industry_Communications_and_Computing(x):
    if x['member_industry'] in['Communications & Computing']:return 1
    else: return 0 
df2['member_industry_Communications_and_Computing']=df.apply(member_industry_Communications_and_Computing, axis=1)
df2.rename(columns={'member_industry_Communications_and_Computing':'member_industry_Communications_&_Computing'}, inplace=True)
#####
def member_industry_Community_and_Social_Services(x):
    if x['member_industry'] in['Community & Social Services']:return 1
    else: return 0 
df2['member_industry_Community_and_Social_Services']=df.apply(member_industry_Community_and_Social_Services, axis=1)
df2.rename(columns={'member_industry_Community_and_Social_Services':'member_industry_Community_&_Social_Services'}, inplace=True)
####
def member_industry_Consultancy_Marketing_and_Legal_services(x):
    if x['member_industry'] in['Consultancy, Marketing & Legal services']:return 1
    else: return 0 
df2['member_industry_Consultancy_Marketing_and_Legal_services']=df.apply(member_industry_Consultancy_Marketing_and_Legal_services, axis=1)
df2.rename(columns={'member_industry_Consultancy_Marketing_and_Legal_services':'member_industry_Consultancy_Marketing_&_Legal_services'}, inplace=True)
#####
def member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property(x):
    if x['member_industry'] in['Dev, Buying & Selling of Domestic or Comm.Property']:return 1
    else: return 0 
df2['member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property']=df.apply(member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property, axis=1)
df2.rename(columns={'member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property':'member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property'}, inplace=True)
#####
def member_industry_Education(x):
    if x['member_industry'] in['Education']:return 1
    else: return 0 
df2['member_industry_Education']=df.apply(member_industry_Education, axis=1)
#####
def member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage(x):
    if x['member_industry'] in['Electricity & Gas Supply, Water, Sewage & Drainage']:return 1
    else: return 0 
df2['member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage']=df.apply(member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage, axis=1)
df2.rename(columns={'member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage':'member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage'}, inplace=True)
####
def member_industry_Food_Textiles_and_Retailing(x):
    if x['member_industry'] in['Food Textiles and Retailing']:return 1
    else: return 0 
df2['member_industry_Food_Textiles_and_Retailing']=df.apply(member_industry_Food_Textiles_and_Retailing, axis=1)
#####
def member_industry_Freight_Transportation_and_Warehousing(x):
    if x['member_industry'] in['Freight Transportation & Warehousing']:return 1
    else: return 0 
df2['member_industry_Freight_Transportation_and_Warehousing']=df.apply(member_industry_Freight_Transportation_and_Warehousing, axis=1)
df2.rename(columns={'member_industry_Freight_Transportation_and_Warehousing':'member_industry_Freight_Transportation_&_Warehousing'}, inplace=True)
#####
def member_industry_Government_Administration(x):
    if x['member_industry'] in['Government Administration']:return 1
    else: return 0 
df2['member_industry_Government_Administration']=df.apply(member_industry_Government_Administration, axis=1)
#####
def member_industry_Health(x):
    if x['member_industry'] in['Health']:return 1
    else: return 0 
df2['member_industry_Health']=df.apply(member_industry_Health, axis=1)
####
def member_industry_Hotels_and_Restaurants(x):
    if x['member_industry'] in['Hotels & Restaurants']:return 1
    else: return 0 
df2['member_industry_Hotels_and_Restaurants']=df.apply(member_industry_Hotels_and_Restaurants, axis=1)
df2.rename(columns={'member_industry_Hotels_and_Restaurants':'member_industry_Hotels_&_Restaurants'}, inplace=True)
#####
def member_industry_Light_Electrical_and_Precision_Manufacturing(x):
    if x['member_industry'] in['Light, Electrical & Precision Manufacturing']:return 1
    else: return 0 
df2['member_industry_Light_Electrical_and_Precision_Manufacturing']=df.apply(member_industry_Light_Electrical_and_Precision_Manufacturing, axis=1)
df2.rename(columns={'member_industry_Light_Electrical_and_Precision_Manufacturing':'member_industry_Light_Electrical_&_Precision_Manufacturing'}, inplace=True)
#####
def member_industry_Manufacturing_and_Retail(x):
    if x['member_industry'] in['Manufacturing and Retail']:return 1
    else: return 0 
df2['member_industry_Manufacturing_and_Retail']=df.apply(member_industry_Manufacturing_and_Retail, axis=1)
#####
def member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_(x):
    if x['member_industry'] in['Mining & Quarrying (incl. Oil & Gas Extraction)']:return 1
    else: return 0 
df2['member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_']=df.apply(member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_, axis=1)
df2.rename(columns={'member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_':'member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_'}, inplace=True)
#####
def member_industry_Miscellaneous(x):
    if x['member_industry'] in['Miscellaneous']:return 1
    else: return 0 
df2['member_industry_Miscellaneous']=df.apply(member_industry_Miscellaneous, axis=1)
#####
def member_industry_Transportation_and_Services(x):
    if x['member_industry'] in['Transportation and Services']:return 1
    else: return 0 
df2['member_industry_Transportation_and_Services']=df.apply(member_industry_Transportation_and_Services, axis=1)
#####
def member_industry_Wholesale(x):
    if x['member_industry'] in['Wholesale']:return 1
    else: return 0 
df2['member_industry_Wholesale']=df.apply(member_industry_Wholesale, axis=1)    
    
    
    
# 'member_location_region'
####
def member_location_region_AFRICA(x):
    if x['member_location_region'] in['AFRICA']:return 1
    else: return 0 
df2['member_location_region_AFRICA']=df.apply(member_location_region_AFRICA, axis=1)
#####
def member_location_region_AMERICAS(x):
    if x['member_location_region'] in['AMERICAS']:return 1
    else: return 0 
df2['member_location_region_AMERICAS']=df.apply(member_location_region_AMERICAS, axis=1)
#####
def member_location_region_ASIA(x):
    if x['member_location_region'] in['ASIA']:return 1
    else: return 0 
df2['member_location_region_ASIA']=df.apply(member_location_region_ASIA, axis=1)

#####
def member_location_region_EUROPE(x):
    if x['member_location_region'] in['EUROPE']:return 1
    else: return 0 
df2['member_location_region_EUROPE']=df.apply(member_location_region_EUROPE, axis=1)

#####
def member_location_region_OCEANIA(x):
    if x['member_location_region'] in['OCEANIA']:return 1
    else: return 0 
df2['member_location_region_OCEANIA']=df.apply(member_location_region_OCEANIA, axis=1)


df2.head()


,member_id,market_most_visits,total_visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount,yf249_Type1Diabetes_indicator,yf249_Type1Diabetes_visits_count,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_indicator,yf250_Type2Diabetes_visits_count,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_indicator,yf272_Dyslipidemia_visits_count,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_indicator,yf401_Hypertension_visits_count,yf401_Hypertension_visits_rate,yf305_TobaccoUse_indicator,yf305_TobaccoUse_visits_count,yf305_TobaccoUse_visits_rate,yf3055_illicitDrugUse_indicator,yf3055_illicitDrugUse_visits_count,yf311_Depression_indicator,yf311_Depression_visits_count,yf311_Depression_visits_rate,yf300_Anxiety_indicator,yf300_Anxiety_visits_count,yf300_Anxiety_visits_rate,yf955_Polypharmcy_indicator,yf955_Polypharmcy_visits_count,yf955_Polypharmcy_visits_rate,yf278_Obesity_indicator,yf278_Obesity_visits_count,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_indicator,yf3002_Kinesiophobia_visits_count,yf3002_Kinesiophobia_visits_rate,yf309_Acopia_indicator,yf309_Acopia_visits_count,yf309_Acopia_visits_rate,total_paid_amount,total_visits_rate,IP_total_visits_rate,OP_total_visits_rate,pppm_obs,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,cost_indicator_obs,yf790_ElevatedPlasmaGlucose_indicator,yf790_ElevatedPlasmaGlucose_visits_count,yf790_ElevatedPlasmaGlucose_visits_rate,yf277_metabolicSyndrome_identifier,yf277_metabolicSyndrome_visits_count,yf277_metabolicSyndrome_visits_rate,yf_metabolicSyndrome_indicator,total_yf_indicator,physiotherapy_proc_indicator,physiotherapy_visits_count,physiotherapy_visits_rate,surgery_proc_indicator,surgery_visits_count,surgery_visits_rate,radio_diag_scans_proc_indicator,radio_diag_scans_visits_count,radio_diag_scans_visits_rate,eval_manag_services_proc_indicator,eval_manag_services_visits_count,eval_manag_services_visits_rate,alternative_medicine_proc_indicator,alternative_medicine_visits_count,alternative_medicine_visits_rate,drugs_related_proc_indicator,drugs_related_visits_count,drugs_related_visits_rate,anesthesia_proc_indicator,anesthesia_visits_count,anesthesia_visits_rate,group_industry_sector_desc,member_family_type,member_nationality,client_location,market_latest_index,member_industry,member_location_region,member_nationality_region,maternity_procedure_indicator,maternity_case_type_indicator,maternity_diagnosis_indicator,all_maternity_indicator,eligible_yrmth_count_perf,total_paid_amount_perf,pppm_perf,cost_indicator_perf,member_gender_F,member_gender_M,member_family_type_C,member_family_type_D,member_family_type_F,member_family_type_S,market_latest_index_market1,market_latest_index_market2,member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_,member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products,member_industry_Communications_&_Computing,member_industry_Community_&_Social_Services,member_industry_Consultancy_Marketing_&_Legal_services,member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property,member_industry_Education,member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage,member_industry_Food_Textiles_and_Retailing,member_industry_Freight_Transportation_&_Warehousing,member_industry_Government_Administration,member_industry_Health,member_industry_Hotels_&_Restaurants,member_industry_Light_Electrical_&_Precision_Manufacturing,member_industry_Manufacturing_and_Retail,member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_,member_industry_Miscellaneous,member_industry_Transportation_and_Services,member_industry_Wholesale,member_location_region_AFRICA,member_location_region_AMERICAS,member_location_region_ASIA,member_location_region_EUROPE,member_location_region_OCEANIA,member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_,member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products,member_industry_Communications_&_Computing,member_industry_

In [110]:
df2.dtypes 

member_id                                                             object
market_most_visits                                                    object
total_visits_count                                                     int64
eligible_yrmth_count                                                   int64
member_location                                                       object
member_age                                                             int64
member_gender                                                         object
IP_visits_count                                                      float64
OP_visits_count                                                      float64
IP_total_paid_amount                                                 float64
OP_total_paid_amount                                                 float64
yf249_Type1Diabetes_indicator                                          int64
yf249_Type1Diabetes_visits_count                                     float64

In [86]:
#fillna for remainder fields
df2[['member_age']]=df2[['member_age']].fillna(0).astype(np.int64)
df2.head()

,member_id,market_most_visits,total_visits_count,eligible_yrmth_count,member_location,member_age,member_gender,IP_visits_count,OP_visits_count,IP_total_paid_amount,OP_total_paid_amount,yf249_Type1Diabetes_indicator,yf249_Type1Diabetes_visits_count,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_indicator,yf250_Type2Diabetes_visits_count,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_indicator,yf272_Dyslipidemia_visits_count,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_indicator,yf401_Hypertension_visits_count,yf401_Hypertension_visits_rate,yf305_TobaccoUse_indicator,yf305_TobaccoUse_visits_count,yf305_TobaccoUse_visits_rate,yf3055_illicitDrugUse_indicator,yf3055_illicitDrugUse_visits_count,yf311_Depression_indicator,yf311_Depression_visits_count,yf311_Depression_visits_rate,yf300_Anxiety_indicator,yf300_Anxiety_visits_count,yf300_Anxiety_visits_rate,yf955_Polypharmcy_indicator,yf955_Polypharmcy_visits_count,yf955_Polypharmcy_visits_rate,yf278_Obesity_indicator,yf278_Obesity_visits_count,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_indicator,yf3002_Kinesiophobia_visits_count,yf3002_Kinesiophobia_visits_rate,yf309_Acopia_indicator,yf309_Acopia_visits_count,yf309_Acopia_visits_rate,total_paid_amount,total_visits_rate,IP_total_visits_rate,OP_total_visits_rate,pppm_obs,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,cost_indicator_obs,yf790_ElevatedPlasmaGlucose_indicator,yf790_ElevatedPlasmaGlucose_visits_count,yf790_ElevatedPlasmaGlucose_visits_rate,yf277_metabolicSyndrome_identifier,yf277_metabolicSyndrome_visits_count,yf277_metabolicSyndrome_visits_rate,yf_metabolicSyndrome_indicator,total_yf_indicator,physiotherapy_proc_indicator,physiotherapy_visits_count,physiotherapy_visits_rate,surgery_proc_indicator,surgery_visits_count,surgery_visits_rate,radio_diag_scans_proc_indicator,radio_diag_scans_visits_count,radio_diag_scans_visits_rate,eval_manag_services_proc_indicator,eval_manag_services_visits_count,eval_manag_services_visits_rate,alternative_medicine_proc_indicator,alternative_medicine_visits_count,alternative_medicine_visits_rate,drugs_related_proc_indicator,drugs_related_visits_count,drugs_related_visits_rate,anesthesia_proc_indicator,anesthesia_visits_count,anesthesia_visits_rate,group_industry_sector_desc,member_family_type,member_nationality,client_location,market_latest_index,member_industry,member_location_region,member_nationality_region,maternity_procedure_indicator,maternity_case_type_indicator,maternity_diagnosis_indicator,all_maternity_indicator,eligible_yrmth_count_perf,total_paid_amount_perf,pppm_perf,cost_indicator_perf,member_gender_F,member_gender_M,member_family_type_C,member_family_type_D,member_family_type_F,member_family_type_S,market_latest_index_market1,market_latest_index_market2,member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_,member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products,member_industry_Communications_&_Computing,member_industry_Community_&_Social_Services,member_industry_Consultancy_Marketing_&_Legal_services,member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property,member_industry_Education,member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage,member_industry_Food_Textiles_and_Retailing,member_industry_Freight_Transportation_&_Warehousing,member_industry_Government_Administration,member_industry_Health,member_industry_Hotels_&_Restaurants,member_industry_Light_Electrical_&_Precision_Manufacturing,member_industry_Manufacturing_and_Retail,member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_,member_industry_Miscellaneous,member_industry_Transportation_and_Services,member_industry_Wholesale,member_location_region_AFRICA,member_location_region_AMERICAS,member_location_region_ASIA,member_location_region_EUROPE,member_location_region_OCEANIA
0,850004357:01,Singapore,13,7,SINGAPORE,57,M,0.0,13.0,0.00,3960.28,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.000,0,0.0,0.0,0,0.0,1,6.0,0.857143,1,5.0,0.714286,1,1.0,0.142857,1,5.0,0.714286,1,

In [87]:
#drop fields
# df3=df2.drop(['member_gender','member_family_type','market_latest_index','member_industry','member_location_region'], axis=1)
# df3.head()
# df2.head(3)

In [88]:
#final set of fields
df3=df2[['member_id','member_age','IP_total_visits_rate', 'OP_total_visits_rate', 'yf249_Type1Diabetes_visits_rate', 'yf250_Type2Diabetes_visits_rate', 'yf272_Dyslipidemia_visits_rate', 'yf401_Hypertension_visits_rate', 'yf305_TobaccoUse_visits_rate', 'yf311_Depression_visits_rate', 'yf300_Anxiety_visits_rate', 'yf955_Polypharmcy_visits_rate', 'yf309_Acopia_visits_rate', 'yf278_Obesity_visits_rate', 'yf3002_Kinesiophobia_visits_rate', 'IP_total_paid_amount_pppm', 'OP_total_paid_amount_pppm', 'physiotherapy_visits_rate', 'surgery_visits_rate', 'radio_diag_scans_visits_rate', 'eval_manag_services_visits_rate', 'alternative_medicine_visits_rate', 'drugs_related_visits_rate', 'anesthesia_visits_rate', 'all_maternity_indicator', 'member_gender_F', 'member_gender_M', 'member_family_type_C', 'member_family_type_D', 'member_family_type_F', 'member_family_type_S', 'market_latest_index_market1', 'market_latest_index_market2', 'member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_', 'member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products', 'member_industry_Communications_&_Computing', 'member_industry_Community_&_Social_Services', 'member_industry_Consultancy_Marketing_&_Legal_services', 'member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property', 'member_industry_Education', 'member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage', 'member_industry_Food_Textiles_and_Retailing', 'member_industry_Freight_Transportation_&_Warehousing', 'member_industry_Government_Administration', 'member_industry_Health', 'member_industry_Hotels_&_Restaurants', 'member_industry_Light_Electrical_&_Precision_Manufacturing', 'member_industry_Manufacturing_and_Retail', 'member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_', 'member_industry_Miscellaneous', 'member_industry_Transportation_and_Services', 'member_industry_Wholesale', 'member_location_region_AFRICA', 'member_location_region_AMERICAS', 'member_location_region_ASIA', 'member_location_region_EUROPE', 'member_location_region_OCEANIA']]
df3.head()

,member_id,member_age,IP_total_visits_rate,OP_total_visits_rate,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_visits_rate,yf305_TobaccoUse_visits_rate,yf311_Depression_visits_rate,yf300_Anxiety_visits_rate,yf955_Polypharmcy_visits_rate,yf309_Acopia_visits_rate,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_visits_rate,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,physiotherapy_visits_rate,surgery_visits_rate,radio_diag_scans_visits_rate,eval_manag_services_visits_rate,alternative_medicine_visits_rate,drugs_related_visits_rate,anesthesia_visits_rate,all_maternity_indicator,member_gender_F,member_gender_M,member_family_type_C,member_family_type_D,member_family_type_F,member_family_type_S,market_latest_index_market1,market_latest_index_market2,member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_,member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products,member_industry_Communications_&_Computing,member_industry_Community_&_Social_Services,member_industry_Consultancy_Marketing_&_Legal_services,member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property,member_industry_Education,member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage,member_industry_Food_Textiles_and_Retailing,member_industry_Freight_Transportation_&_Warehousing,member_industry_Government_Administration,member_industry_Health,member_industry_Hotels_&_Restaurants,member_industry_Light_Electrical_&_Precision_Manufacturing,member_industry_Manufacturing_and_Retail,member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_,member_industry_Miscellaneous,member_industry_Transportation_and_Services,member_industry_Wholesale,member_location_region_AFRICA,member_location_region_AMERICAS,member_location_region_ASIA,member_location_region_EUROPE,member_location_region_OCEANIA
0,850004357:01,57,0.00,1.857143,0.0,0.0,0.0,0.000,0.0,0.857143,0.714286,0.142857,0.0,0.714286,0.857143,565.754286,565.754286,0.0,0.0,142.857143,1000.0,285.714286,714.285714,0.0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,850004357:02,56,0.00,3.300000,0.0,0.0,0.0,0.000,0.0,2.000000,1.800000,0.200000,0.0,6.100000,1.800000,564.870000,564.870000,1000.0,0.0,100.000000,1200.0,500.000000,600.000000,0.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,850004965:01,58,0.00,1.500000,0.0,0.0,0.0,0.000,0.0,0.500000,0.000000,0.833333,0.0,2.333333,1.333333,496.643333,496.643333,0.0,0.0,166.666667,1000.0,0.000000,1500.000000,0.0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,850009036:01,55,0.25,4.375000,0.0,0.0,0.5,0.125,0.0,1.000000,0.875000,0.375000,0.0,3.250000,1.375000,2284.108750,2284.108750,625.0,125.0,625.000000,2750.0,0.000000,1875.000000,0.0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,850012059:02,47,0.10,3.500000,0.0,0.0,0.0,0.000,0.0,0.000000,0.000000,2.200000,0.0,1.700000,0.000000,3996.219000,3996.219000,2000.0,200.0,300.000000,1800.0,0.000000,900.000000,100.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [89]:
#data type check
# df.dtypes
# df3.dtypes

In [90]:
# turn the memberid from integer to string (ideally, it should have always been a string to begin with)
#     df3['member_id'] = df3['member_id'].astype('str')


In [91]:
# turn the memberid into an index
final = df3.set_index('member_id')
final.head()

,member_age,IP_total_visits_rate,OP_total_visits_rate,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_visits_rate,yf305_TobaccoUse_visits_rate,yf311_Depression_visits_rate,yf300_Anxiety_visits_rate,yf955_Polypharmcy_visits_rate,yf309_Acopia_visits_rate,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_visits_rate,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,physiotherapy_visits_rate,surgery_visits_rate,radio_diag_scans_visits_rate,eval_manag_services_visits_rate,alternative_medicine_visits_rate,drugs_related_visits_rate,anesthesia_visits_rate,all_maternity_indicator,member_gender_F,member_gender_M,member_family_type_C,member_family_type_D,member_family_type_F,member_family_type_S,market_latest_index_market1,market_latest_index_market2,member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_,member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products,member_industry_Communications_&_Computing,member_industry_Community_&_Social_Services,member_industry_Consultancy_Marketing_&_Legal_services,member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property,member_industry_Education,member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage,member_industry_Food_Textiles_and_Retailing,member_industry_Freight_Transportation_&_Warehousing,member_industry_Government_Administration,member_industry_Health,member_industry_Hotels_&_Restaurants,member_industry_Light_Electrical_&_Precision_Manufacturing,member_industry_Manufacturing_and_Retail,member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_,member_industry_Miscellaneous,member_industry_Transportation_and_Services,member_industry_Wholesale,member_location_region_AFRICA,member_location_region_AMERICAS,member_location_region_ASIA,member_location_region_EUROPE,member_location_region_OCEANIA
member_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
850004357:01,57,0.00,1.857143,0.0,0.0,0.0,0.000,0.0,0.857143,0.714286,0.142857,0.0,0.714286,0.857143,565.754286,565.754286,0.0,0.0,142.857143,1000.0,285.714286,714.285714,0.0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850004357:02,56,0.00,3.300000,0.0,0.0,0.0,0.000,0.0,2.000000,1.800000,0.200000,0.0,6.100000,1.800000,564.870000,564.870000,1000.0,0.0,100.000000,1200.0,500.000000,600.000000,0.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850004965:01,58,0.00,1.500000,0.0,0.0,0.0,0.000,0.0,0.500000,0.000000,0.833333,0.0,2.333333,1.333333,496.643333,496.643333,0.0,0.0,166.666667,1000.0,0.000000,1500.000000,0.0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
850009036:01,55,0.25,4.375000,0.0,0.0,0.5,0.125,0.0,1.000000,0.875000,0.375000,0.0,3.250000,1.375000,2284.108750,2284.108750,625.0,125.0,625.000000,2750.0,0.000000,1875.000000,0.0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850012059:02,47,0.10,3.500000,0.0,0.0,0.0,0.000,0.0,0.000000,0.000000,2.200000,0.0,1.700000,0.000000,3996.219000,3996.219000,2000.0,200.0,300.000000,1800.0,0.000000,900.000000,100.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [92]:
final.head()

,member_age,IP_total_visits_rate,OP_total_visits_rate,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_visits_rate,yf305_TobaccoUse_visits_rate,yf311_Depression_visits_rate,yf300_Anxiety_visits_rate,yf955_Polypharmcy_visits_rate,yf309_Acopia_visits_rate,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_visits_rate,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,physiotherapy_visits_rate,surgery_visits_rate,radio_diag_scans_visits_rate,eval_manag_services_visits_rate,alternative_medicine_visits_rate,drugs_related_visits_rate,anesthesia_visits_rate,all_maternity_indicator,member_gender_F,member_gender_M,member_family_type_C,member_family_type_D,member_family_type_F,member_family_type_S,market_latest_index_market1,market_latest_index_market2,member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_,member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products,member_industry_Communications_&_Computing,member_industry_Community_&_Social_Services,member_industry_Consultancy_Marketing_&_Legal_services,member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property,member_industry_Education,member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage,member_industry_Food_Textiles_and_Retailing,member_industry_Freight_Transportation_&_Warehousing,member_industry_Government_Administration,member_industry_Health,member_industry_Hotels_&_Restaurants,member_industry_Light_Electrical_&_Precision_Manufacturing,member_industry_Manufacturing_and_Retail,member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_,member_industry_Miscellaneous,member_industry_Transportation_and_Services,member_industry_Wholesale,member_location_region_AFRICA,member_location_region_AMERICAS,member_location_region_ASIA,member_location_region_EUROPE,member_location_region_OCEANIA
member_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
850004357:01,57,0.00,1.857143,0.0,0.0,0.0,0.000,0.0,0.857143,0.714286,0.142857,0.0,0.714286,0.857143,565.754286,565.754286,0.0,0.0,142.857143,1000.0,285.714286,714.285714,0.0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850004357:02,56,0.00,3.300000,0.0,0.0,0.0,0.000,0.0,2.000000,1.800000,0.200000,0.0,6.100000,1.800000,564.870000,564.870000,1000.0,0.0,100.000000,1200.0,500.000000,600.000000,0.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850004965:01,58,0.00,1.500000,0.0,0.0,0.0,0.000,0.0,0.500000,0.000000,0.833333,0.0,2.333333,1.333333,496.643333,496.643333,0.0,0.0,166.666667,1000.0,0.000000,1500.000000,0.0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
850009036:01,55,0.25,4.375000,0.0,0.0,0.5,0.125,0.0,1.000000,0.875000,0.375000,0.0,3.250000,1.375000,2284.108750,2284.108750,625.0,125.0,625.000000,2750.0,0.000000,1875.000000,0.0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850012059:02,47,0.10,3.500000,0.0,0.0,0.0,0.000,0.0,0.000000,0.000000,2.200000,0.0,1.700000,0.000000,3996.219000,3996.219000,2000.0,200.0,300.000000,1800.0,0.000000,900.000000,100.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [93]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8859 entries, 850004357:01 to 858027801:02
Data columns (total 56 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   member_age                                                         8859 non-null   int64  
 1   IP_total_visits_rate                                               8859 non-null   float64
 2   OP_total_visits_rate                                               8859 non-null   float64
 3   yf249_Type1Diabetes_visits_rate                                    8859 non-null   float64
 4   yf250_Type2Diabetes_visits_rate                                    8859 non-null   float64
 5   yf272_Dyslipidemia_visits_rate                                     8859 non-null   float64
 6   yf401_Hypertension_visits_rate                                     8859 non-null   float64
 7   yf305_Toba

In [94]:
final.dtypes

member_age                                                             int64
IP_total_visits_rate                                                 float64
OP_total_visits_rate                                                 float64
yf249_Type1Diabetes_visits_rate                                      float64
yf250_Type2Diabetes_visits_rate                                      float64
yf272_Dyslipidemia_visits_rate                                       float64
yf401_Hypertension_visits_rate                                       float64
yf305_TobaccoUse_visits_rate                                         float64
yf311_Depression_visits_rate                                         float64
yf300_Anxiety_visits_rate                                            float64
yf955_Polypharmcy_visits_rate                                        float64
yf309_Acopia_visits_rate                                             float64
yf278_Obesity_visits_rate                                            float64

In [95]:
final.isnull().sum()

member_age                                                           0
IP_total_visits_rate                                                 0
OP_total_visits_rate                                                 0
yf249_Type1Diabetes_visits_rate                                      0
yf250_Type2Diabetes_visits_rate                                      0
yf272_Dyslipidemia_visits_rate                                       0
yf401_Hypertension_visits_rate                                       0
yf305_TobaccoUse_visits_rate                                         0
yf311_Depression_visits_rate                                         0
yf300_Anxiety_visits_rate                                            0
yf955_Polypharmcy_visits_rate                                        0
yf309_Acopia_visits_rate                                             0
yf278_Obesity_visits_rate                                            0
yf3002_Kinesiophobia_visits_rate                                     0
IP_tot

In [96]:
# # export
# wr.s3.to_csv(
#     df=final,
#     path='s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/msk_global_sg_hk_index_preprocessing_ABHI_Date_Final.csv')

In [97]:
# #import table to view
# df = pd.read_csv('s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/msk_global_sg_hk_index_preprocessing_ABHI_Date_Final.csv')
# df.head(3)

In [98]:
# real time feature
# the latest date of data (for rename fields to indicate latest)
msk_elig_claim["incurred__date__year_month_"].head(3)


0          NaT
1          NaT
2   2023-12-04
Name: incurred__date__year_month_, dtype: datetime64[ns]

In [99]:
# real time feature
# the latest date of data (for rename fields to indicate latest)
latest_date_name = msk_elig_claim["incurred__date__year_month_"].max().strftime("%m%d%Y")
print("The latest date got from the source: " + str(latest_date_name))

The latest date got from the source: 12282023


In [100]:
from dateutil.relativedelta import relativedelta

In [101]:
# 1 year before index date
year_before_index_date_name = (msk_elig_claim["incurred__date__year_month_"].max() - relativedelta(months=3)) - relativedelta(months=12)

print("1 year before index date: " + str(year_before_index_date_name)) 

1 year before index date: 2022-09-28 00:00:00
